# Retail Banking Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Retail Banking Challenge 2: Credit Default Prediction**.

**Goal**: Predict `DefaultLabel` (0/1) for each customer-week combination
**Metric**: Macro-F1 - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular customer panel data with a simple Random Forest classifier.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769",        # Get from your team dashboard
    team_name="synergy-minds"     # Your exact team name
)
# 1. Load all data sources
print("📂 Loading all data sources...")

# Core customer data
customers = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
devices = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/devices_all.csv")

# Transaction data
transactions_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
transactions_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")

# Panel data
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

# Device sessions
with open("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json", 'r') as f:
    device_sessions = pd.json_normalize(json.load(f))

print("✅ Data loaded successfully!")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

# Load all data
print("📂 Loading data...")
customers = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
transactions_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
transactions_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

with open("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json", 'r') as f:
    device_sessions = pd.json_normalize(json.load(f))

print("🎯 Creating advanced temporal and behavioral features...")

def create_customer_segmentation_features(customers, accounts):
    """Create customer segmentation and financial health features"""
    features = customers.copy()

    # Encode categorical variables
    le_city = LabelEncoder()
    features['HomeCity_encoded'] = le_city.fit_transform(features['HomeCity'].fillna('Unknown'))

    # Account-based features
    account_stats = accounts.groupby('CustomerID').agg({
        'Balance': ['sum', 'mean', 'std', 'max'],
        'Limit': ['sum', 'max'],
        'AccountID': 'count',
        'Type': lambda x: (x == 'credit_card').sum()
    }).reset_index()
    account_stats.columns = ['CustomerID', 'total_balance', 'avg_balance', 'std_balance', 'max_balance',
                            'total_limit', 'max_limit', 'num_accounts', 'num_credit_cards']

    # Credit utilization
    account_stats['overall_utilization'] = np.where(
        account_stats['total_limit'] > 0,
        account_stats['total_balance'] / account_stats['total_limit'],
        0
    )

    # Account type distribution
    account_types = pd.get_dummies(accounts[['CustomerID', 'Type']], columns=['Type'], prefix='account')
    account_types = account_types.groupby('CustomerID').sum().reset_index()

    # Merge features
    features = features.merge(account_stats, on='CustomerID', how='left')
    features = features.merge(account_types, on='CustomerID', how='left')

    # Financial health indicators
    features['balance_to_salary'] = features['total_balance'] / (features['AnnualSalary'] + 1)
    features['limit_to_salary'] = features['total_limit'] / (features['AnnualSalary'] + 1)

    # Risk flags
    features['high_utilization_flag'] = (features['overall_utilization'] > 0.8).astype(int)
    features['multiple_credit_cards'] = (features['num_credit_cards'] > 2).astype(int)
    features['low_credit_score'] = (features['CreditScore'] < 600).astype(int)

    # Customer segmentation
    features['premium_customer'] = ((features['AnnualSalary'] > features['AnnualSalary'].quantile(0.7)) &
                                   (features['CreditScore'] > 700)).astype(int)
    features['risky_customer'] = ((features['CreditScore'] < 580) |
                                 (features['overall_utilization'] > 0.9)).astype(int)

    # Financial health score (FIXED: This was missing)
    features['financial_health_score'] = (
        (features['CreditScore'] / 850) * 0.4 +
        (1 - features['overall_utilization'].clip(0, 1)) * 0.3 +
        (1 - (features['balance_to_salary'].clip(0, 2) / 2)) * 0.3
    )

    return features.fillna(0)

def create_temporal_sequence_features(panel_df):
    """Create time-series features from panel data"""
    temporal_features = []

    for customer_id in panel_df['CustomerID'].unique():
        customer_data = panel_df[panel_df['CustomerID'] == customer_id].sort_values('Week')

        if len(customer_data) > 1:
            # Rolling statistics
            for window in [3, 5]:
                customer_data[f'utilization_ma_{window}'] = customer_data['Utilisation'].rolling(window=window, min_periods=1).mean()
                customer_data[f'payment_ratio_ma_{window}'] = customer_data['PaymentRatio'].rolling(window=window, min_periods=1).mean()
                customer_data[f'utilization_std_{window}'] = customer_data['Utilisation'].rolling(window=window, min_periods=1).std()

            # Trends and changes
            customer_data['utilization_trend'] = customer_data['Utilisation'].diff().rolling(window=3, min_periods=1).mean()
            customer_data['payment_ratio_trend'] = customer_data['PaymentRatio'].diff().rolling(window=3, min_periods=1).mean()

            # Volatility measures
            customer_data['utilization_volatility'] = customer_data['Utilisation'].rolling(window=4, min_periods=1).std()
            customer_data['payment_volatility'] = customer_data['PaymentRatio'].rolling(window=4, min_periods=1).std()

            # Cumulative risk indicators
            customer_data['cumulative_inquiries'] = customer_data['HardInquiries'].cumsum()
            customer_data['high_utilization_weeks'] = (customer_data['Utilisation'] > 0.7).cumsum()
            customer_data['low_payment_weeks'] = (customer_data['PaymentRatio'] < 0.2).cumsum()

            # Recent deterioration indicators
            customer_data['recent_utilization_increase'] = (customer_data['Utilisation'] > customer_data['Utilisation'].shift(1)).astype(int)
            customer_data['recent_payment_decrease'] = (customer_data['PaymentRatio'] < customer_data['PaymentRatio'].shift(1)).astype(int)

        else:
            # Single week - use current values
            customer_data['utilization_ma_3'] = customer_data['Utilisation']
            customer_data['payment_ratio_ma_3'] = customer_data['PaymentRatio']
            customer_data['utilization_std_3'] = 0
            customer_data['utilization_trend'] = 0
            customer_data['payment_ratio_trend'] = 0
            customer_data['utilization_volatility'] = 0
            customer_data['payment_volatility'] = 0
            customer_data['cumulative_inquiries'] = customer_data['HardInquiries']
            customer_data['high_utilization_weeks'] = (customer_data['Utilisation'] > 0.7).astype(int)
            customer_data['low_payment_weeks'] = (customer_data['PaymentRatio'] < 0.2).astype(int)
            customer_data['recent_utilization_increase'] = 0
            customer_data['recent_payment_decrease'] = 0

        temporal_features.append(customer_data)

    return pd.concat(temporal_features, ignore_index=True).fillna(0)

def create_transaction_behavior_features(transactions_df):
    """Create comprehensive transaction behavior features"""
    transactions = transactions_df.copy()
    transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'], format='ISO8601')

    # Basic transaction stats
    txn_stats = transactions.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': ['sum', 'mean', 'std', 'max', 'min'],
        'Timestamp': ['min', 'max', 'nunique']
    }).reset_index()
    txn_stats.columns = ['CustomerID', 'txn_count', 'total_amount', 'avg_amount', 'amount_std',
                        'max_amount', 'min_amount', 'first_txn', 'last_txn', 'txn_days']

    # Transaction period and velocity
    txn_stats['txn_period_days'] = (txn_stats['last_txn'] - txn_stats['first_txn']).dt.days + 1
    txn_stats['daily_txn_rate'] = txn_stats['txn_count'] / txn_stats['txn_period_days']
    txn_stats['daily_spending'] = txn_stats['total_amount'] / txn_stats['txn_period_days']

    # Large transaction behavior
    large_txn_threshold = transactions['Amount'].quantile(0.85)
    large_txns = transactions[transactions['Amount'] > large_txn_threshold]
    large_txn_stats = large_txns.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': 'mean'
    }).reset_index()
    large_txn_stats.columns = ['CustomerID', 'large_txn_count', 'avg_large_txn']

    # Channel preferences
    channel_dummies = pd.get_dummies(transactions[['CustomerID', 'Channel']],
                                   columns=['Channel'], prefix='channel')
    channel_means = channel_dummies.groupby('CustomerID').mean().reset_index()

    # MCC category spending
    if 'MCC_Group' in transactions.columns:
        mcc_dummies = pd.get_dummies(transactions[['CustomerID', 'MCC_Group']],
                                   columns=['MCC_Group'], prefix='mcc')
        mcc_means = mcc_dummies.groupby('CustomerID').mean().reset_index()
    else:
        mcc_means = pd.DataFrame(columns=['CustomerID'])

    # Temporal patterns
    transactions['hour'] = transactions['Timestamp'].dt.hour
    transactions['day_of_week'] = transactions['Timestamp'].dt.dayofweek
    transactions['is_weekend'] = (transactions['day_of_week'] >= 5).astype(int)
    transactions['is_night'] = ((transactions['hour'] >= 22) | (transactions['hour'] <= 6)).astype(int)

    temporal_patterns = transactions.groupby('CustomerID').agg({
        'is_weekend': 'mean',
        'is_night': 'mean',
        'hour': ['mean', 'std']
    }).reset_index()
    temporal_patterns.columns = ['CustomerID', 'weekend_txn_ratio', 'night_txn_ratio',
                                'avg_txn_hour', 'std_txn_hour']

    # Merge all transaction features
    features = txn_stats.merge(large_txn_stats, on='CustomerID', how='left')
    features = features.merge(channel_means, on='CustomerID', how='left')
    if 'CustomerID' in mcc_means.columns:
        features = features.merge(mcc_means, on='CustomerID', how='left')
    features = features.merge(temporal_patterns, on='CustomerID', how='left')

    # Create risk scores
    features['large_txn_ratio'] = features['large_txn_count'] / (features['txn_count'] + 1)
    features['spending_volatility'] = features['amount_std'] / (features['avg_amount'] + 1)
    features['transaction_consistency'] = features['txn_days'] / (features['txn_period_days'] + 1)

    features['spending_risk_score'] = (
        features['large_txn_ratio'] * 0.4 +
        features['spending_volatility'] * 0.3 +
        features['night_txn_ratio'] * 0.3
    )

    return features.fillna(0)

def create_session_risk_features(device_sessions):
    """Create session-based risk features"""
    sessions = device_sessions.copy()
    sessions['Timestamp'] = pd.to_datetime(sessions['Timestamp'], format='ISO8601')

    # Basic session stats
    session_stats = sessions.groupby('CustomerID').agg({
        'SessionID': 'count',
        'City': 'nunique',
        'DeviceID': 'nunique',
        'Timestamp': ['min', 'max']
    }).reset_index()
    session_stats.columns = ['CustomerID', 'session_count', 'unique_cities', 'unique_devices',
                            'first_session', 'last_session']

    # Session frequency
    session_stats['session_period_days'] = (session_stats['last_session'] - session_stats['first_session']).dt.days + 1
    session_stats['daily_sessions'] = session_stats['session_count'] / session_stats['session_period_days']

    # Action analysis
    def analyze_actions(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'total_actions': len(actions_list),
                'financial_actions': 0,
                'financial_amount': 0,
                'logins': 0,
                'sensitive_actions': 0
            }
            for action in actions_list:
                if isinstance(action, dict):
                    action_type = action.get('type', '')
                    if action_type in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['financial_amount'] += action.get('amount', 0)
                    if action_type in ['transfer', 'payment', 'account_view']:
                        stats['sensitive_actions'] += 1
                elif action == 'login':
                    stats['logins'] += 1
            return stats
        return {'total_actions': 0, 'financial_actions': 0, 'financial_amount': 0,
                'logins': 0, 'sensitive_actions': 0}

    action_stats = sessions['Actions'].apply(analyze_actions)
    action_df = pd.DataFrame(action_stats.tolist())
    action_df['CustomerID'] = sessions['CustomerID'].values

    # Aggregate action stats
    action_agg = action_df.groupby('CustomerID').agg({
        'total_actions': 'sum',
        'financial_actions': 'sum',
        'financial_amount': 'sum',
        'logins': 'sum',
        'sensitive_actions': 'sum'
    }).reset_index()

    # Calculate ratios
    action_agg['financial_action_ratio'] = action_agg['financial_actions'] / (action_agg['total_actions'] + 1)
    action_agg['sensitive_action_ratio'] = action_agg['sensitive_actions'] / (action_agg['total_actions'] + 1)
    action_agg['avg_financial_amount'] = action_agg['financial_amount'] / (action_agg['financial_actions'] + 1)

    # Merge session features
    features = session_stats.merge(action_agg, on='CustomerID', how='left')

    # Session risk scores
    features['geographic_risk'] = (features['unique_cities'] > 3).astype(int)
    features['device_risk'] = (features['unique_devices'] > 2).astype(int)
    features['activity_risk'] = (features['daily_sessions'] > features['daily_sessions'].quantile(0.8)).astype(int)

    features['session_risk_score'] = (
        features['geographic_risk'] * 0.4 +
        features['device_risk'] * 0.3 +
        features['sensitive_action_ratio'] * 0.3
    )

    return features.fillna(0)

print("🔄 Creating advanced feature sets...")

# Create all feature sets
customer_features = create_customer_segmentation_features(customers, accounts)
temporal_train = create_temporal_sequence_features(train_panel)
temporal_test = create_temporal_sequence_features(test_panel)
transaction_features = create_transaction_behavior_features(pd.concat([transactions_train, transactions_test]))
session_features = create_session_risk_features(device_sessions)

print("✅ Advanced features created:")
print(f"   Customer features: {customer_features.shape}")
print(f"   Temporal features train: {temporal_train.shape}")
print(f"   Temporal features test: {temporal_test.shape}")
print(f"   Transaction features: {transaction_features.shape}")
print(f"   Session features: {session_features.shape}")

# Combine all features
def create_final_features(panel_df, customer_df, temporal_df, transaction_df, session_df):
    """Combine all feature sources"""
    # Start with temporal panel
    features = temporal_df.copy()

    # Add customer features
    features = features.merge(customer_df, on='CustomerID', how='left')

    # Add transaction features
    features = features.merge(transaction_df, on='CustomerID', how='left')

    # Add session features
    features = features.merge(session_df, on='CustomerID', how='left')

    # Create powerful interaction features (FIXED: Check if columns exist)
    risk_components = []

    # Check each component exists before using it
    if 'utilization_ma_3' in features.columns:
        risk_components.append(features['utilization_ma_3'] * 0.2)

    if 'payment_ratio_ma_3' in features.columns:
        risk_components.append((1 - features['payment_ratio_ma_3']) * 0.2)

    if 'financial_health_score' in features.columns:
        risk_components.append((1 - features['financial_health_score']) * 0.2)

    if 'spending_risk_score' in features.columns:
        risk_components.append(features['spending_risk_score'] * 0.2)

    if 'session_risk_score' in features.columns:
        risk_components.append(features['session_risk_score'] * 0.2)

    # Create comprehensive risk score if we have components
    if risk_components:
        features['comprehensive_risk_score'] = sum(risk_components) / len(risk_components)
    else:
        features['comprehensive_risk_score'] = 0

    # Deterioration score
    deterioration_components = []
    if 'utilization_trend' in features.columns:
        deterioration_components.append(features['utilization_trend'] * 0.4)

    if 'recent_payment_decrease' in features.columns:
        deterioration_components.append(features['recent_payment_decrease'] * 0.3)

    if 'cumulative_inquiries' in features.columns:
        deterioration_components.append((features['cumulative_inquiries'] / 10) * 0.3)

    if deterioration_components:
        features['deterioration_score'] = sum(deterioration_components)
    else:
        features['deterioration_score'] = 0

    # Financial stress indicators
    stress_components = []
    if 'overall_utilization' in features.columns:
        stress_components.append((features['overall_utilization'] > 0.8).astype(int) * 0.4)

    if 'balance_to_salary' in features.columns:
        stress_components.append((features['balance_to_salary'] > 0.5).astype(int) * 0.3)

    if 'CreditScore' in features.columns:
        stress_components.append((features['CreditScore'] < 600).astype(int) * 0.3)

    if stress_components:
        features['financial_stress'] = sum(stress_components)
    else:
        features['financial_stress'] = 0

    return features.fillna(0)

# Create final datasets
X_train_final = create_final_features(train_panel, customer_features, temporal_train, transaction_features, session_features)
X_test_final = create_final_features(test_panel, customer_features, temporal_test, transaction_features, session_features)

# Prepare for modeling
y_train = X_train_final['DefaultLabel'].astype(int)
non_feature_cols = ['CustomerID', 'Week', 'DefaultLabel']
feature_cols = [col for col in X_train_final.columns if col not in non_feature_cols]

X_train = X_train_final[feature_cols]
X_test = X_test_final[feature_cols]

print(f"🎯 Final dataset: {X_train.shape[1]} features, {X_train.shape[0]} samples")

# Ensure numeric types
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)

# Feature selection - keep top 40 features
print("🔍 Selecting top features...")
k = min(40, X_train.shape[1])
selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

selected_features = [feature_cols[i] for i in selector.get_support(indices=True)]

print(f"   Selected {len(selected_features)} most predictive features")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

print("🤖 Training highly optimized model...")

# Optimized Random Forest
model = RandomForestClassifier(
    n_estimators=400,
    max_depth=25,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=0.7,
    class_weight='balanced_subsample',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

# Feature importance
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("📊 Top 15 most important features:")
print(feature_importance.head(15))

# Generate predictions with multiple threshold options
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Try different thresholds and analyze
threshold_options = [0.15, 0.2, 0.25, 0.3, 0.35]
best_threshold = 0.25

print("🎯 Analyzing threshold options:")
for threshold in threshold_options:
    preds = (y_pred_proba > threshold).astype(int)
    default_rate = preds.mean()
    print(f"   Threshold {threshold}: Default rate = {default_rate:.3f} ({preds.sum()} defaults)")

    # Prefer thresholds that give 5-12% default rate (typical for credit risk)
    if 0.05 <= default_rate <= 0.12:
        best_threshold = threshold

print(f"✅ Selected threshold: {best_threshold}")

# Final predictions
predictions = (y_pred_proba > best_threshold).astype(int)

# Create submission
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': predictions
})

submission_df.to_csv("retailbanking_challenge2_final_predictions.csv", index=False)

print(f"✅ Final predictions saved: {submission_df.shape[0]} predictions")
print(f"   Default rate: {predictions.mean():.3f} ({predictions.sum()} defaults)")

# Submit
try:
    from agentds import BenchmarkClient
    client = BenchmarkClient(api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769", team_name="synergy-minds")

    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_final_predictions.csv")

    if result['success']:
        print("🎉 Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        if result['score'] < 0.9:
            print(f"   🎯 Need improvement: {0.931 - result['score']:.4f} to reach top score")
        else:
            print("   🏆 Excellent score!")
    else:
        print("❌ Submission failed!")

except Exception as e:
    print(f"💥 Submission error: {e}")

print("\n💡 Key improvements in this approach:")
print("   • Temporal patterns and trends from panel data")
print("   • Comprehensive risk scoring from multiple sources")
print("   • Advanced behavioral features from transactions and sessions")
print("   • Optimized feature selection and model parameters")

📂 Loading data...
🎯 Creating advanced temporal and behavioral features...
🔄 Creating advanced feature sets...
✅ Advanced features created:
   Customer features: (1000, 27)
   Temporal features train: (13301, 21)
   Temporal features test: (13290, 20)
   Transaction features: (1000, 32)
   Session features: (1000, 20)
🎯 Final dataset: 97 features, 13301 samples
🔍 Selecting top features...
   Selected 40 most predictive features
🤖 Training highly optimized model...
📊 Top 15 most important features:
                     feature  importance
35             total_actions    0.214580
6                CreditScore    0.214220
37    sensitive_action_ratio    0.122896
17    financial_health_score    0.065299
1              HardInquiries    0.056378
5          low_payment_weeks    0.042956
8                total_limit    0.036592
14           limit_to_salary    0.029893
24           txn_period_days    0.029396
36    financial_action_ratio    0.028632
9                  max_limit    0.024105
31    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

print("🚀 Loading and preparing data for elite performance...")

# Load data
customers = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
transactions_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
transactions_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

with open("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json", 'r') as f:
    device_sessions = pd.json_normalize(json.load(f))

print("🎯 Creating elite feature engineering pipeline...")

def create_elite_customer_features(customers, accounts):
    """Create sophisticated customer segmentation and risk profiling"""
    features = customers.copy()

    # Advanced encoding
    le_city = LabelEncoder()
    features['HomeCity_encoded'] = le_city.fit_transform(features['HomeCity'].fillna('Unknown'))

    # Account portfolio analysis
    account_metrics = accounts.groupby('CustomerID').agg({
        'AccountID': 'count',
        'Balance': ['sum', 'mean', 'std', 'max', 'min', 'median'],
        'Limit': ['sum', 'max', 'mean'],
        'Type': lambda x: x.nunique()
    }).reset_index()
    account_metrics.columns = ['CustomerID', 'total_accounts', 'balance_sum', 'balance_mean',
                              'balance_std', 'balance_max', 'balance_min', 'balance_median',
                              'limit_sum', 'limit_max', 'limit_mean', 'account_type_diversity']

    # Credit-specific features
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_metrics = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['sum', 'mean', 'max'],
            'Limit': ['sum', 'max', 'mean']
        }).reset_index()
        credit_metrics.columns = ['CustomerID', 'credit_balance_sum', 'credit_balance_mean',
                                 'credit_balance_max', 'credit_limit_sum', 'credit_limit_max',
                                 'credit_limit_mean']

        credit_metrics['credit_utilization'] = credit_metrics['credit_balance_sum'] / (credit_metrics['credit_limit_sum'] + 1)
        credit_metrics['max_credit_utilization'] = credit_metrics['credit_balance_max'] / (credit_metrics['credit_limit_max'] + 1)
    else:
        credit_metrics = pd.DataFrame(columns=['CustomerID', 'credit_utilization', 'max_credit_utilization'])

    # Account type composition
    account_composition = pd.get_dummies(accounts[['CustomerID', 'Type']], columns=['Type'], prefix='account')
    account_composition = account_composition.groupby('CustomerID').sum().reset_index()

    # Merge features
    features = features.merge(account_metrics, on='CustomerID', how='left')
    if 'CustomerID' in credit_metrics.columns:
        features = features.merge(credit_metrics, on='CustomerID', how='left')
    features = features.merge(account_composition, on='CustomerID', how='left')

    # Advanced financial ratios
    features['balance_to_salary'] = features['balance_sum'] / (features['AnnualSalary'] + 1)
    features['limit_to_salary'] = features['limit_sum'] / (features['AnnualSalary'] + 1)
    features['credit_depth'] = features['credit_limit_sum'] / (features['AnnualSalary'] + 1)

    # Risk profiling
    features['high_risk_profile'] = (
        (features['CreditScore'] < 580).astype(int) * 0.4 +
        (features.get('credit_utilization', 0) > 0.8).astype(int) * 0.3 +
        (features['balance_to_salary'] > 0.5).astype(int) * 0.3
    )

    # Customer lifetime value proxy
    features['clv_score'] = (
        (features['AnnualSalary'] / features['AnnualSalary'].max()) * 0.4 +
        (features['CreditScore'] / 850) * 0.3 +
        (features['Tenure'] / features['Tenure'].max()) * 0.3
    )

    return features.fillna(0)

def create_temporal_dynamics_features(panel_df):
    """Create sophisticated time-series features with trend analysis"""
    temporal_features = []

    for customer_id in panel_df['CustomerID'].unique():
        cust_data = panel_df[panel_df['CustomerID'] == customer_id].sort_values('Week')

        # Rolling statistics with multiple windows
        for window in [2, 3, 4]:
            cust_data[f'utilization_ma_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).mean()
            cust_data[f'payment_ratio_ma_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).mean()
            cust_data[f'utilization_std_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).std()
            cust_data[f'payment_ratio_std_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).std()

        # Trend analysis
        cust_data['utilization_trend_3'] = cust_data['Utilisation'].diff(periods=2).fillna(0)
        cust_data['payment_trend_3'] = cust_data['PaymentRatio'].diff(periods=2).fillna(0)

        # Acceleration (second derivative)
        cust_data['utilization_acceleration'] = cust_data['utilization_trend_3'].diff().fillna(0)
        cust_data['payment_acceleration'] = cust_data['payment_trend_3'].diff().fillna(0)

        # Volatility measures
        cust_data['utilization_volatility'] = cust_data['Utilisation'].rolling(4, min_periods=1).std()
        cust_data['payment_volatility'] = cust_data['PaymentRatio'].rolling(4, min_periods=1).std()

        # Behavioral patterns
        cust_data['high_utilization_streak'] = (cust_data['Utilisation'] > 0.7).astype(int)
        cust_data['low_payment_streak'] = (cust_data['PaymentRatio'] < 0.2).astype(int)

        # Calculate streaks
        for i in range(1, len(cust_data)):
            if cust_data.iloc[i]['high_utilization_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('high_utilization_streak')] = \
                    cust_data.iloc[i-1]['high_utilization_streak'] + 1
            if cust_data.iloc[i]['low_payment_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('low_payment_streak')] = \
                    cust_data.iloc[i-1]['low_payment_streak'] + 1

        # Deterioration indicators
        cust_data['financial_deterioration'] = (
            (cust_data['utilization_trend_3'] > 0).astype(int) * 0.5 +
            (cust_data['payment_trend_3'] < 0).astype(int) * 0.5
        )

        temporal_features.append(cust_data)

    return pd.concat(temporal_features, ignore_index=True).fillna(0)

def create_advanced_transaction_features(transactions_df):
    """Create comprehensive transaction behavior profiling"""
    transactions = transactions_df.copy()
    transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'], format='ISO8601')

    # Basic transaction metrics
    txn_metrics = transactions.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': ['sum', 'mean', 'std', 'max', 'min', 'median', 'skew'],
        'Timestamp': ['min', 'max', 'nunique']
    }).reset_index()
    txn_metrics.columns = ['CustomerID', 'txn_count', 'amount_sum', 'amount_mean', 'amount_std',
                          'amount_max', 'amount_min', 'amount_median', 'amount_skew',
                          'first_txn', 'last_txn', 'active_days']

    # Transaction period and velocity
    txn_metrics['txn_period_days'] = (txn_metrics['last_txn'] - txn_metrics['first_txn']).dt.days + 1
    txn_metrics['daily_txn_frequency'] = txn_metrics['txn_count'] / txn_metrics['txn_period_days']
    txn_metrics['daily_spending'] = txn_metrics['amount_sum'] / txn_metrics['txn_period_days']

    # Large transaction analysis
    large_txn_threshold = transactions['Amount'].quantile(0.8)
    large_txns = transactions[transactions['Amount'] > large_txn_threshold]

    if not large_txns.empty:
        large_txn_stats = large_txns.groupby('CustomerID').agg({
            'TxnID': 'count',
            'Amount': ['mean', 'sum', 'max']
        }).reset_index()
        large_txn_stats.columns = ['CustomerID', 'large_txn_count', 'large_txn_avg',
                                  'large_txn_sum', 'large_txn_max']

        large_txn_stats['large_txn_ratio'] = large_txn_stats['large_txn_count'] / txn_metrics['txn_count']
        large_txn_stats['large_amount_ratio'] = large_txn_stats['large_txn_sum'] / txn_metrics['amount_sum']
    else:
        large_txn_stats = pd.DataFrame(columns=['CustomerID', 'large_txn_ratio', 'large_amount_ratio'])

    # Channel behavior
    channel_behavior = pd.get_dummies(transactions[['CustomerID', 'Channel']],
                                    columns=['Channel'], prefix='channel')
    channel_behavior = channel_behavior.groupby('CustomerID').mean().reset_index()

    # MCC spending patterns
    if 'MCC_Group' in transactions.columns:
        mcc_behavior = pd.get_dummies(transactions[['CustomerID', 'MCC_Group']],
                                    columns=['MCC_Group'], prefix='mcc')
        mcc_behavior = mcc_behavior.groupby('CustomerID').mean().reset_index()
    else:
        mcc_behavior = pd.DataFrame(columns=['CustomerID'])

    # Temporal patterns
    transactions['hour'] = transactions['Timestamp'].dt.hour
    transactions['day_of_week'] = transactions['Timestamp'].dt.dayofweek
    transactions['is_weekend'] = (transactions['day_of_week'] >= 5).astype(int)
    transactions['is_night'] = ((transactions['hour'] >= 22) | (transactions['hour'] <= 6)).astype(int)

    temporal_patterns = transactions.groupby('CustomerID').agg({
        'is_weekend': 'mean',
        'is_night': 'mean',
        'hour': ['mean', 'std', lambda x: x.mode()[0] if len(x.mode()) > 0 else 12]
    }).reset_index()
    temporal_patterns.columns = ['CustomerID', 'weekend_ratio', 'night_ratio',
                                'avg_txn_hour', 'std_txn_hour', 'mode_txn_hour']

    # Spending consistency
    daily_spending = transactions.groupby([transactions['Timestamp'].dt.date, 'CustomerID'])['Amount'].sum().reset_index()
    spending_consistency = daily_spending.groupby('CustomerID')['Amount'].agg(['mean', 'std']).reset_index()
    spending_consistency.columns = ['CustomerID', 'daily_spending_mean', 'daily_spending_std']
    spending_consistency['spending_volatility'] = spending_consistency['daily_spending_std'] / (spending_consistency['daily_spending_mean'] + 1)

    # Merge all features
    features = txn_metrics.merge(channel_behavior, on='CustomerID', how='left')
    if 'CustomerID' in mcc_behavior.columns:
        features = features.merge(mcc_behavior, on='CustomerID', how='left')
    features = features.merge(temporal_patterns, on='CustomerID', how='left')
    features = features.merge(spending_consistency, on='CustomerID', how='left')

    if 'CustomerID' in large_txn_stats.columns:
        features = features.merge(large_txn_stats, on='CustomerID', how='left')

    # Risk scores
    features['transaction_risk_score'] = (
        features.get('large_txn_ratio', 0) * 0.3 +
        features['spending_volatility'] * 0.3 +
        features['night_ratio'] * 0.2 +
        (features['amount_skew'].abs() * 0.2)
    )

    return features.fillna(0)

def create_session_intelligence_features(device_sessions):
    """Create advanced session behavior intelligence"""
    sessions = device_sessions.copy()
    sessions['Timestamp'] = pd.to_datetime(sessions['Timestamp'], format='ISO8601')

    # Session frequency and patterns
    session_freq = sessions.groupby('CustomerID').agg({
        'SessionID': 'count',
        'City': 'nunique',
        'DeviceID': 'nunique',
        'IP': 'nunique',
        'Timestamp': ['min', 'max']
    }).reset_index()
    session_freq.columns = ['CustomerID', 'session_count', 'cities_visited', 'devices_used',
                           'ips_used', 'first_session', 'last_session']

    session_freq['session_period_days'] = (session_freq['last_session'] - session_freq['first_session']).dt.days + 1
    session_freq['daily_sessions'] = session_freq['session_count'] / session_freq['session_period_days']

    # Advanced action analysis
    def analyze_advanced_actions(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'total_actions': len(actions_list),
                'financial_actions': 0,
                'sensitive_actions': 0,
                'login_count': 0,
                'transfer_amount': 0,
                'payment_amount': 0,
                'unique_action_types': set()
            }

            for action in actions_list:
                if isinstance(action, dict):
                    action_type = action.get('type', '')
                    stats['unique_action_types'].add(action_type)

                    if action_type in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                        amount = action.get('amount', 0)
                        if action_type == 'transfer':
                            stats['transfer_amount'] += amount
                        else:
                            stats['payment_amount'] += amount
                    elif action_type == 'account_view':
                        stats['sensitive_actions'] += 1
                elif action == 'login':
                    stats['login_count'] += 1
                    stats['unique_action_types'].add('login')
                elif action == 'logout':
                    stats['unique_action_types'].add('logout')

            stats['unique_action_count'] = len(stats['unique_action_types'])
            return stats
        return {'total_actions': 0, 'financial_actions': 0, 'sensitive_actions': 0,
                'login_count': 0, 'transfer_amount': 0, 'payment_amount': 0, 'unique_action_count': 0}

    action_analysis = sessions['Actions'].apply(analyze_advanced_actions)
    action_df = pd.DataFrame(action_analysis.tolist())
    action_df['CustomerID'] = sessions['CustomerID'].values

    # Aggregate action intelligence
    action_intel = action_df.groupby('CustomerID').agg({
        'total_actions': 'sum',
        'financial_actions': 'sum',
        'sensitive_actions': 'sum',
        'login_count': 'sum',
        'transfer_amount': 'sum',
        'payment_amount': 'sum',
        'unique_action_count': 'mean'
    }).reset_index()

    # Calculate behavioral ratios
    action_intel['financial_action_ratio'] = action_intel['financial_actions'] / (action_intel['total_actions'] + 1)
    action_intel['sensitive_action_ratio'] = action_intel['sensitive_actions'] / (action_intel['total_actions'] + 1)
    action_intel['login_frequency'] = action_intel['login_count'] / session_freq['session_count']
    action_intel['avg_financial_amount'] = (action_intel['transfer_amount'] + action_intel['payment_amount']) / (action_intel['financial_actions'] + 1)

    # Merge features
    features = session_freq.merge(action_intel, on='CustomerID', how='left')

    # Security and risk indicators
    features['geographic_dispersion'] = features['cities_visited'] / (features['session_count'] + 1)
    features['device_diversity'] = features['devices_used'] / (features['session_count'] + 1)
    features['ip_diversity'] = features['ips_used'] / (features['session_count'] + 1)

    features['session_risk_score'] = (
        features['geographic_dispersion'] * 0.3 +
        features['device_diversity'] * 0.3 +
        features['sensitive_action_ratio'] * 0.4
    )

    return features.fillna(0)

print("🔄 Building elite feature sets...")

# Create all feature sets
customer_features = create_elite_customer_features(customers, accounts)
temporal_train = create_temporal_dynamics_features(train_panel)
temporal_test = create_temporal_dynamics_features(test_panel)

# Combine train and test transactions for consistent feature engineering
all_transactions = pd.concat([transactions_train, transactions_test])
transaction_features = create_advanced_transaction_features(all_transactions)
session_features = create_session_intelligence_features(device_sessions)

print("🎯 Creating final feature matrix...")

def create_elite_feature_matrix(panel_df, customer_df, temporal_df, transaction_df, session_df):
    """Combine all elite features with intelligent interactions"""
    features = temporal_df.copy()

    # Merge all feature sources
    features = features.merge(customer_df, on='CustomerID', how='left')
    features = features.merge(transaction_df, on='CustomerID', how='left')
    features = features.merge(session_df, on='CustomerID', how='left')

    # Create powerful interaction features
    # Financial stress indicators
    stress_components = []
    if 'utilization_ma_3' in features.columns:
        stress_components.append(features['utilization_ma_3'] * 0.25)
    if 'payment_ratio_ma_3' in features.columns:
        stress_components.append((1 - features['payment_ratio_ma_3']) * 0.25)
    if 'high_risk_profile' in features.columns:
        stress_components.append(features['high_risk_profile'] * 0.25)
    if 'transaction_risk_score' in features.columns:
        stress_components.append(features['transaction_risk_score'] * 0.25)

    if stress_components:
        features['comprehensive_stress_score'] = sum(stress_components)

    # Behavioral risk indicators
    behavior_components = []
    if 'session_risk_score' in features.columns:
        behavior_components.append(features['session_risk_score'] * 0.4)
    if 'financial_deterioration' in features.columns:
        behavior_components.append(features['financial_deterioration'] * 0.3)
    if 'spending_volatility' in features.columns:
        behavior_components.append(features['spending_volatility'] * 0.3)

    if behavior_components:
        features['behavioral_risk_score'] = sum(behavior_components)

    # Credit capacity indicators
    if 'CreditScore' in features.columns and 'credit_utilization' in features.columns:
        features['credit_health_index'] = (
            (features['CreditScore'] / 850) * 0.6 +
            (1 - features['credit_utilization'].clip(0, 1)) * 0.4
        )

    # Payment behavior indicators
    if 'payment_trend_3' in features.columns and 'payment_volatility' in features.columns:
        features['payment_behavior_score'] = (
            (1 - features['payment_trend_3'].clip(-1, 0).abs()) * 0.5 +
            (1 - features['payment_volatility'].clip(0, 1)) * 0.5
        )

    return features.fillna(0)

# Create final datasets
X_train_elite = create_elite_feature_matrix(train_panel, customer_features, temporal_train, transaction_features, session_features)
X_test_elite = create_elite_feature_matrix(test_panel, customer_features, temporal_test, transaction_features, session_features)

# Prepare for modeling
y_train = X_train_elite['DefaultLabel'].astype(int)
non_feature_cols = ['CustomerID', 'Week', 'DefaultLabel', 'first_txn', 'last_txn', 'first_session', 'last_session']
feature_cols = [col for col in X_train_elite.columns if col not in non_feature_cols]

X_train = X_train_elite[feature_cols]
X_test = X_test_elite[feature_cols]

print(f"✅ Elite feature matrix: {X_train.shape[1]} features, {X_train.shape[0]} samples")

# Ensure numeric types and handle infinite values
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)

print("🔍 Performing elite feature selection...")

# Use Random Forest for feature selection
selector_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
selector_model.fit(X_train, y_train)

# Select top features based on importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': selector_model.feature_importances_
}).sort_values('importance', ascending=False)

# Keep top 50 features
top_features = feature_importance.head(50)['feature'].tolist()
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(f"🎯 Selected {len(top_features)} most predictive features")

print("🤖 Training elite ensemble model...")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Train optimized ensemble
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight='balanced_subsample',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

gb_model = GradientBoostingClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# Train both models
rf_model.fit(X_train_scaled, y_train)
gb_model.fit(X_train_scaled, y_train)

# Ensemble predictions
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
gb_proba = gb_model.predict_proba(X_test_scaled)[:, 1]

# Weighted ensemble (favor RF more as it typically performs better)
ensemble_proba = rf_proba * 0.7 + gb_proba * 0.3

print("📊 Analyzing optimal threshold...")

# Find optimal threshold for Macro-F1
thresholds = np.arange(0.1, 0.5, 0.05)
default_rates = []

for threshold in thresholds:
    preds = (ensemble_proba > threshold).astype(int)
    default_rates.append(preds.mean())

# Select threshold that gives reasonable default rate (5-15%)
optimal_threshold = 0.25
for i, threshold in enumerate(thresholds):
    if 0.05 <= default_rates[i] <= 0.15:
        optimal_threshold = threshold
        break

print(f"✅ Optimal threshold: {optimal_threshold:.2f}")
print(f"   Expected default rate: {default_rates[thresholds.tolist().index(optimal_threshold)]:.3f}")

# Final predictions
final_predictions = (ensemble_proba > optimal_threshold).astype(int)

# Create submission
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': final_predictions
})

submission_df.to_csv("retailbanking_challenge2_elite_predictions.csv", index=False)

print(f"🎉 Elite predictions saved: {len(final_predictions)} predictions")
print(f"   Default rate: {final_predictions.mean():.3f} ({final_predictions.sum()} defaults)")

# Submit predictions
try:
    from agentds import BenchmarkClient
    client = BenchmarkClient(api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769", team_name="synergy-minds")

    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_elite_predictions.csv")

    if result['success']:
        print("🏆 Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        current_score = result['score']
        if current_score < 0.9:
            improvement_needed = 0.931 - current_score
            print(f"   🎯 Need improvement: {improvement_needed:.4f} to reach top score")
        else:
            print("   💪 Excellent! You're in the top tier!")
    else:
        print("❌ Submission failed!")

except Exception as e:
    print(f"💥 Submission error: {e}")

print("\n💡 Elite strategy features:")
print("   • Advanced temporal dynamics with trend analysis")
print("   • Sophisticated customer segmentation and risk profiling")
print("   • Comprehensive transaction behavior intelligence")
print("   • Session-based security and risk indicators")
print("   • Ensemble modeling with optimized threshold selection")
print("   • Elite feature selection focusing on predictive power")

🚀 Loading and preparing data for elite performance...
🎯 Creating elite feature engineering pipeline...
🔄 Building elite feature sets...
🎯 Creating final feature matrix...
✅ Elite feature matrix: 118 features, 13301 samples
🔍 Performing elite feature selection...
🎯 Selected 50 most predictive features
🤖 Training elite ensemble model...
📊 Analyzing optimal threshold...
✅ Optimal threshold: 0.10
   Expected default rate: 0.137
🎉 Elite predictions saved: 13290 predictions
   Default rate: 0.137 (1826 defaults)
✅ Prediction submitted successfully!
📊 Score: 0.9185 (Macro-F1)
✅ Validation passed
🏆 Submission successful!
   📊 Score: 0.9185
   💪 Excellent! You're in the top tier!

💡 Elite strategy features:
   • Advanced temporal dynamics with trend analysis
   • Sophisticated customer segmentation and risk profiling
   • Comprehensive transaction behavior intelligence
   • Session-based security and risk indicators
   • Ensemble modeling with optimized threshold selection
   • Elite feature se

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("XGBoost not available, using alternative models")

print("🚀 Loading and preparing data for elite performance...")

# Load data
customers = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
transactions_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
transactions_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

with open("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json", 'r') as f:
    import json
    device_sessions = pd.json_normalize(json.load(f))

print("🎯 Creating elite feature engineering pipeline...")

def create_elite_customer_features(customers, accounts):
    """Create sophisticated customer segmentation and risk profiling"""
    features = customers.copy()

    # Advanced encoding
    le_city = LabelEncoder()
    features['HomeCity_encoded'] = le_city.fit_transform(features['HomeCity'].fillna('Unknown'))

    # Account portfolio analysis
    account_metrics = accounts.groupby('CustomerID').agg({
        'AccountID': 'count',
        'Balance': ['sum', 'mean', 'std', 'max', 'min', 'median'],
        'Limit': ['sum', 'max', 'mean'],
        'Type': lambda x: x.nunique()
    }).reset_index()
    account_metrics.columns = ['CustomerID', 'total_accounts', 'balance_sum', 'balance_mean',
                              'balance_std', 'balance_max', 'balance_min', 'balance_median',
                              'limit_sum', 'limit_max', 'limit_mean', 'account_type_diversity']

    # Credit-specific features
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_metrics = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['sum', 'mean', 'max'],
            'Limit': ['sum', 'max', 'mean']
        }).reset_index()
        credit_metrics.columns = ['CustomerID', 'credit_balance_sum', 'credit_balance_mean',
                                 'credit_balance_max', 'credit_limit_sum', 'credit_limit_max',
                                 'credit_limit_mean']

        credit_metrics['credit_utilization'] = credit_metrics['credit_balance_sum'] / (credit_metrics['credit_limit_sum'] + 1)
        credit_metrics['max_credit_utilization'] = credit_metrics['credit_balance_max'] / (credit_metrics['credit_limit_max'] + 1)
    else:
        credit_metrics = pd.DataFrame(columns=['CustomerID', 'credit_utilization', 'max_credit_utilization'])

    # Account type composition
    account_composition = pd.get_dummies(accounts[['CustomerID', 'Type']], columns=['Type'], prefix='account')
    account_composition = account_composition.groupby('CustomerID').sum().reset_index()

    # Merge features
    features = features.merge(account_metrics, on='CustomerID', how='left')
    if 'CustomerID' in credit_metrics.columns:
        features = features.merge(credit_metrics, on='CustomerID', how='left')
    features = features.merge(account_composition, on='CustomerID', how='left')

    # Advanced financial ratios
    features['balance_to_salary'] = features['balance_sum'] / (features['AnnualSalary'] + 1)
    features['limit_to_salary'] = features['limit_sum'] / (features['AnnualSalary'] + 1)
    features['credit_depth'] = features['credit_limit_sum'] / (features['AnnualSalary'] + 1)

    # Risk profiling
    features['high_risk_profile'] = (
        (features['CreditScore'] < 580).astype(int) * 0.4 +
        (features.get('credit_utilization', 0) > 0.8).astype(int) * 0.3 +
        (features['balance_to_salary'] > 0.5).astype(int) * 0.3
    )

    # Customer lifetime value proxy
    features['clv_score'] = (
        (features['AnnualSalary'] / features['AnnualSalary'].max()) * 0.4 +
        (features['CreditScore'] / 850) * 0.3 +
        (features['Tenure'] / features['Tenure'].max()) * 0.3
    )

    return features.fillna(0)

def enhance_temporal_features(panel_df):
    """Add more sophisticated time-series features"""
    features = panel_df.copy()

    # Seasonal decomposition
    features['utilization_seasonal'] = features.groupby('CustomerID')['Utilisation'].transform(
        lambda x: x - x.rolling(4, min_periods=1).mean()
    )

    # Change point detection
    features['utilization_change_point'] = (
        features['Utilisation'].rolling(3).std() > features['Utilisation'].rolling(6).std() * 1.5
    ).astype(int)

    # Momentum indicators
    features['utilization_momentum'] = features['Utilisation'] - features['Utilisation'].shift(2)
    features['payment_momentum'] = features['PaymentRatio'] - features['PaymentRatio'].shift(2)

    return features

def create_temporal_dynamics_features(panel_df):
    """Create sophisticated time-series features with trend analysis"""
    temporal_features = []

    for customer_id in panel_df['CustomerID'].unique():
        cust_data = panel_df[panel_df['CustomerID'] == customer_id].sort_values('Week')

        # Rolling statistics with multiple windows
        for window in [2, 3, 4]:
            cust_data[f'utilization_ma_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).mean()
            cust_data[f'payment_ratio_ma_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).mean()
            cust_data[f'utilization_std_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).std()
            cust_data[f'payment_ratio_std_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).std()

        # Trend analysis
        cust_data['utilization_trend_3'] = cust_data['Utilisation'].diff(periods=2).fillna(0)
        cust_data['payment_trend_3'] = cust_data['PaymentRatio'].diff(periods=2).fillna(0)

        # Acceleration (second derivative)
        cust_data['utilization_acceleration'] = cust_data['utilization_trend_3'].diff().fillna(0)
        cust_data['payment_acceleration'] = cust_data['payment_trend_3'].diff().fillna(0)

        # Volatility measures
        cust_data['utilization_volatility'] = cust_data['Utilisation'].rolling(4, min_periods=1).std()
        cust_data['payment_volatility'] = cust_data['PaymentRatio'].rolling(4, min_periods=1).std()

        # Behavioral patterns
        cust_data['high_utilization_streak'] = (cust_data['Utilisation'] > 0.7).astype(int)
        cust_data['low_payment_streak'] = (cust_data['PaymentRatio'] < 0.2).astype(int)

        # Calculate streaks
        for i in range(1, len(cust_data)):
            if cust_data.iloc[i]['high_utilization_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('high_utilization_streak')] = \
                    cust_data.iloc[i-1]['high_utilization_streak'] + 1
            if cust_data.iloc[i]['low_payment_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('low_payment_streak')] = \
                    cust_data.iloc[i-1]['low_payment_streak'] + 1

        # Deterioration indicators
        cust_data['financial_deterioration'] = (
            (cust_data['utilization_trend_3'] > 0).astype(int) * 0.5 +
            (cust_data['payment_trend_3'] < 0).astype(int) * 0.5
        )

        temporal_features.append(cust_data)

    result = pd.concat(temporal_features, ignore_index=True).fillna(0)

    # Apply enhanced temporal features
    result = enhance_temporal_features(result)

    return result

def create_advanced_transaction_features(transactions_df):
    """Create comprehensive transaction behavior profiling"""
    transactions = transactions_df.copy()
    transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'], format='ISO8601')

    # Basic transaction metrics
    txn_metrics = transactions.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': ['sum', 'mean', 'std', 'max', 'min', 'median', 'skew'],
        'Timestamp': ['min', 'max', 'nunique']
    }).reset_index()
    txn_metrics.columns = ['CustomerID', 'txn_count', 'amount_sum', 'amount_mean', 'amount_std',
                          'amount_max', 'amount_min', 'amount_median', 'amount_skew',
                          'first_txn', 'last_txn', 'active_days']

    # Transaction period and velocity
    txn_metrics['txn_period_days'] = (txn_metrics['last_txn'] - txn_metrics['first_txn']).dt.days + 1
    txn_metrics['daily_txn_frequency'] = txn_metrics['txn_count'] / txn_metrics['txn_period_days']
    txn_metrics['daily_spending'] = txn_metrics['amount_sum'] / txn_metrics['txn_period_days']

    # Large transaction analysis
    large_txn_threshold = transactions['Amount'].quantile(0.8)
    large_txns = transactions[transactions['Amount'] > large_txn_threshold]

    if not large_txns.empty:
        large_txn_stats = large_txns.groupby('CustomerID').agg({
            'TxnID': 'count',
            'Amount': ['mean', 'sum', 'max']
        }).reset_index()
        large_txn_stats.columns = ['CustomerID', 'large_txn_count', 'large_txn_avg',
                                  'large_txn_sum', 'large_txn_max']

        large_txn_stats['large_txn_ratio'] = large_txn_stats['large_txn_count'] / txn_metrics['txn_count']
        large_txn_stats['large_amount_ratio'] = large_txn_stats['large_txn_sum'] / txn_metrics['amount_sum']
    else:
        large_txn_stats = pd.DataFrame(columns=['CustomerID', 'large_txn_ratio', 'large_amount_ratio'])

    # Channel behavior
    channel_behavior = pd.get_dummies(transactions[['CustomerID', 'Channel']],
                                    columns=['Channel'], prefix='channel')
    channel_behavior = channel_behavior.groupby('CustomerID').mean().reset_index()

    # MCC spending patterns
    if 'MCC_Group' in transactions.columns:
        mcc_behavior = pd.get_dummies(transactions[['CustomerID', 'MCC_Group']],
                                    columns=['MCC_Group'], prefix='mcc')
        mcc_behavior = mcc_behavior.groupby('CustomerID').mean().reset_index()
    else:
        mcc_behavior = pd.DataFrame(columns=['CustomerID'])

    # Temporal patterns
    transactions['hour'] = transactions['Timestamp'].dt.hour
    transactions['day_of_week'] = transactions['Timestamp'].dt.dayofweek
    transactions['is_weekend'] = (transactions['day_of_week'] >= 5).astype(int)
    transactions['is_night'] = ((transactions['hour'] >= 22) | (transactions['hour'] <= 6)).astype(int)

    temporal_patterns = transactions.groupby('CustomerID').agg({
        'is_weekend': 'mean',
        'is_night': 'mean',
        'hour': ['mean', 'std', lambda x: x.mode()[0] if len(x.mode()) > 0 else 12]
    }).reset_index()
    temporal_patterns.columns = ['CustomerID', 'weekend_ratio', 'night_ratio',
                                'avg_txn_hour', 'std_txn_hour', 'mode_txn_hour']

    # Spending consistency
    daily_spending = transactions.groupby([transactions['Timestamp'].dt.date, 'CustomerID'])['Amount'].sum().reset_index()
    spending_consistency = daily_spending.groupby('CustomerID')['Amount'].agg(['mean', 'std']).reset_index()
    spending_consistency.columns = ['CustomerID', 'daily_spending_mean', 'daily_spending_std']
    spending_consistency['spending_volatility'] = spending_consistency['daily_spending_std'] / (spending_consistency['daily_spending_mean'] + 1)

    # Merge all features
    features = txn_metrics.merge(channel_behavior, on='CustomerID', how='left')
    if 'CustomerID' in mcc_behavior.columns:
        features = features.merge(mcc_behavior, on='CustomerID', how='left')
    features = features.merge(temporal_patterns, on='CustomerID', how='left')
    features = features.merge(spending_consistency, on='CustomerID', how='left')

    if 'CustomerID' in large_txn_stats.columns:
        features = features.merge(large_txn_stats, on='CustomerID', how='left')

    # Risk scores
    features['transaction_risk_score'] = (
        features.get('large_txn_ratio', 0) * 0.3 +
        features['spending_volatility'] * 0.3 +
        features['night_ratio'] * 0.2 +
        (features['amount_skew'].abs() * 0.2)
    )

    return features.fillna(0)

def create_session_intelligence_features(device_sessions):
    """Create advanced session behavior intelligence"""
    sessions = device_sessions.copy()
    sessions['Timestamp'] = pd.to_datetime(sessions['Timestamp'], format='ISO8601')

    # Session frequency and patterns
    session_freq = sessions.groupby('CustomerID').agg({
        'SessionID': 'count',
        'City': 'nunique',
        'DeviceID': 'nunique',
        'IP': 'nunique',
        'Timestamp': ['min', 'max']
    }).reset_index()
    session_freq.columns = ['CustomerID', 'session_count', 'cities_visited', 'devices_used',
                           'ips_used', 'first_session', 'last_session']

    session_freq['session_period_days'] = (session_freq['last_session'] - session_freq['first_session']).dt.days + 1
    session_freq['daily_sessions'] = session_freq['session_count'] / session_freq['session_period_days']

    # Advanced action analysis
    def analyze_advanced_actions(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'total_actions': len(actions_list),
                'financial_actions': 0,
                'sensitive_actions': 0,
                'login_count': 0,
                'transfer_amount': 0,
                'payment_amount': 0,
                'unique_action_types': set()
            }

            for action in actions_list:
                if isinstance(action, dict):
                    action_type = action.get('type', '')
                    stats['unique_action_types'].add(action_type)

                    if action_type in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                        amount = action.get('amount', 0)
                        if action_type == 'transfer':
                            stats['transfer_amount'] += amount
                        else:
                            stats['payment_amount'] += amount
                    elif action_type == 'account_view':
                        stats['sensitive_actions'] += 1
                elif action == 'login':
                    stats['login_count'] += 1
                    stats['unique_action_types'].add('login')
                elif action == 'logout':
                    stats['unique_action_types'].add('logout')

            stats['unique_action_count'] = len(stats['unique_action_types'])
            return stats
        return {'total_actions': 0, 'financial_actions': 0, 'sensitive_actions': 0,
                'login_count': 0, 'transfer_amount': 0, 'payment_amount': 0, 'unique_action_count': 0}

    action_analysis = sessions['Actions'].apply(analyze_advanced_actions)
    action_df = pd.DataFrame(action_analysis.tolist())
    action_df['CustomerID'] = sessions['CustomerID'].values

    # Aggregate action intelligence
    action_intel = action_df.groupby('CustomerID').agg({
        'total_actions': 'sum',
        'financial_actions': 'sum',
        'sensitive_actions': 'sum',
        'login_count': 'sum',
        'transfer_amount': 'sum',
        'payment_amount': 'sum',
        'unique_action_count': 'mean'
    }).reset_index()

    # Calculate behavioral ratios
    action_intel['financial_action_ratio'] = action_intel['financial_actions'] / (action_intel['total_actions'] + 1)
    action_intel['sensitive_action_ratio'] = action_intel['sensitive_actions'] / (action_intel['total_actions'] + 1)
    action_intel['login_frequency'] = action_intel['login_count'] / session_freq['session_count']
    action_intel['avg_financial_amount'] = (action_intel['transfer_amount'] + action_intel['payment_amount']) / (action_intel['financial_actions'] + 1)

    # Merge features
    features = session_freq.merge(action_intel, on='CustomerID', how='left')

    # Security and risk indicators
    features['geographic_dispersion'] = features['cities_visited'] / (features['session_count'] + 1)
    features['device_diversity'] = features['devices_used'] / (features['session_count'] + 1)
    features['ip_diversity'] = features['ips_used'] / (features['session_count'] + 1)

    features['session_risk_score'] = (
        features['geographic_dispersion'] * 0.3 +
        features['device_diversity'] * 0.3 +
        features['sensitive_action_ratio'] * 0.4
    )

    return features.fillna(0)

def create_interaction_features(features):
    """Create powerful interaction terms"""
    # Financial stress interactions
    if all(col in features.columns for col in ['utilization_ma_3', 'payment_ratio_ma_3']):
        features['utilization_payment_interaction'] = (
            features['utilization_ma_3'] * (1 - features['payment_ratio_ma_3'])
        )

    # Credit-behavior interactions
    if all(col in features.columns for col in ['CreditScore', 'transaction_risk_score']):
        features['credit_behavior_risk'] = (
            (1 - features['CreditScore'] / 850) * features['transaction_risk_score']
        )

    # Multi-dimensional risk scoring
    risk_components = []
    risk_cols = ['high_risk_profile', 'transaction_risk_score', 'session_risk_score', 'financial_deterioration']

    for col in risk_cols:
        if col in features.columns:
            risk_components.append(features[col])

    if risk_components:
        features['comprehensive_risk_index'] = sum(risk_components) / len(risk_components)

    return features

print("🔄 Building elite feature sets...")

# Create all feature sets
customer_features = create_elite_customer_features(customers, accounts)
temporal_train = create_temporal_dynamics_features(train_panel)
temporal_test = create_temporal_dynamics_features(test_panel)

# Combine train and test transactions for consistent feature engineering
all_transactions = pd.concat([transactions_train, transactions_test])
transaction_features = create_advanced_transaction_features(all_transactions)
session_features = create_session_intelligence_features(device_sessions)

print("🎯 Creating final feature matrix...")

def create_elite_feature_matrix(panel_df, customer_df, temporal_df, transaction_df, session_df):
    """Combine all elite features with intelligent interactions"""
    features = temporal_df.copy()

    # Merge all feature sources
    features = features.merge(customer_df, on='CustomerID', how='left')
    features = features.merge(transaction_df, on='CustomerID', how='left')
    features = features.merge(session_df, on='CustomerID', how='left')

    # Create powerful interaction features
    # Financial stress indicators
    stress_components = []
    if 'utilization_ma_3' in features.columns:
        stress_components.append(features['utilization_ma_3'] * 0.25)
    if 'payment_ratio_ma_3' in features.columns:
        stress_components.append((1 - features['payment_ratio_ma_3']) * 0.25)
    if 'high_risk_profile' in features.columns:
        stress_components.append(features['high_risk_profile'] * 0.25)
    if 'transaction_risk_score' in features.columns:
        stress_components.append(features['transaction_risk_score'] * 0.25)

    if stress_components:
        features['comprehensive_stress_score'] = sum(stress_components)

    # Behavioral risk indicators
    behavior_components = []
    if 'session_risk_score' in features.columns:
        behavior_components.append(features['session_risk_score'] * 0.4)
    if 'financial_deterioration' in features.columns:
        behavior_components.append(features['financial_deterioration'] * 0.3)
    if 'spending_volatility' in features.columns:
        behavior_components.append(features['spending_volatility'] * 0.3)

    if behavior_components:
        features['behavioral_risk_score'] = sum(behavior_components)

    # Credit capacity indicators
    if 'CreditScore' in features.columns and 'credit_utilization' in features.columns:
        features['credit_health_index'] = (
            (features['CreditScore'] / 850) * 0.6 +
            (1 - features['credit_utilization'].clip(0, 1)) * 0.4
        )

    # Payment behavior indicators
    if 'payment_trend_3' in features.columns and 'payment_volatility' in features.columns:
        features['payment_behavior_score'] = (
            (1 - features['payment_trend_3'].clip(-1, 0).abs()) * 0.5 +
            (1 - features['payment_volatility'].clip(0, 1)) * 0.5
        )

    # Apply interaction features
    features = create_interaction_features(features)

    return features.fillna(0)

# Create final datasets
X_train_elite = create_elite_feature_matrix(train_panel, customer_features, temporal_train, transaction_features, session_features)
X_test_elite = create_elite_feature_matrix(test_panel, customer_features, temporal_test, transaction_features, session_features)

# Prepare for modeling
y_train = X_train_elite['DefaultLabel'].astype(int)
non_feature_cols = ['CustomerID', 'Week', 'DefaultLabel', 'first_txn', 'last_txn', 'first_session', 'last_session']
feature_cols = [col for col in X_train_elite.columns if col not in non_feature_cols]

X_train = X_train_elite[feature_cols]
X_test = X_test_elite[feature_cols]

print(f"✅ Elite feature matrix: {X_train.shape[1]} features, {X_train.shape[0]} samples")

# Ensure numeric types and handle infinite values
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)

print("🔍 Performing elite feature selection...")

# Use Random Forest for feature selection
selector_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
selector_model.fit(X_train, y_train)

# Select top features based on importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': selector_model.feature_importances_
}).sort_values('importance', ascending=False)

# Keep top 50 features
top_features = feature_importance.head(50)['feature'].tolist()
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(f"🎯 Selected {len(top_features)} most predictive features")

def optimize_threshold_with_business_rules(y_true, y_proba, min_default_rate=0.05, max_default_rate=0.15):
    """Optimize threshold considering business constraints"""
    thresholds = np.linspace(0.1, 0.5, 100)
    best_threshold = 0.25
    best_f1 = 0

    for threshold in thresholds:
        y_pred = (y_proba > threshold).astype(int)
        default_rate = y_pred.mean()

        # Apply business constraints
        if min_default_rate <= default_rate <= max_default_rate:
            f1 = f1_score(y_true, y_pred, average='macro')

            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

    return best_threshold, best_f1

def create_stacked_ensemble(X_train, y_train, X_test):
    """Create stacked ensemble with multiple base models"""
    base_models = {
        'rf': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
        'gb': GradientBoostingClassifier(n_estimators=150, random_state=42)
    }

    # Add XGBoost if available
    if XGB_AVAILABLE:
        base_models['xgb'] = xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    # Create meta-features using cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    meta_train = np.zeros((X_train.shape[0], len(base_models)))
    meta_test = np.zeros((X_test.shape[0], len(base_models)))

    for i, (name, model) in enumerate(base_models.items()):
        fold_predictions = np.zeros(X_train.shape[0])
        test_predictions = []

        for train_idx, val_idx in skf.split(X_train, y_train):
            model.fit(X_train[train_idx], y_train[train_idx])
            fold_predictions[val_idx] = model.predict_proba(X_train[val_idx])[:, 1]
            test_predictions.append(model.predict_proba(X_test)[:, 1])

        meta_train[:, i] = fold_predictions
        meta_test[:, i] = np.mean(test_predictions, axis=0)

    # Meta-model
    meta_model = LogisticRegression()
    meta_model.fit(meta_train, y_train)

    return meta_model.predict_proba(meta_test)[:, 1]

def temporal_cross_validation(model, X, y, weeks, n_splits=5):
    """Time-aware cross-validation"""
    unique_weeks = sorted(weeks.unique())
    split_size = len(unique_weeks) // n_splits

    scores = []

    for i in range(n_splits):
        val_weeks = unique_weeks[i * split_size:(i + 1) * split_size]

        train_mask = ~weeks.isin(val_weeks)
        val_mask = weeks.isin(val_weeks)

        X_train_fold, X_val = X[train_mask], X[val_mask]
        y_train_fold, y_val = y[train_mask], y[val_mask]

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict_proba(X_val)[:, 1]

        # Optimize threshold for this fold
        threshold, _ = optimize_threshold_with_business_rules(y_val, y_pred)
        val_pred = (y_pred > threshold).astype(int)

        score = f1_score(y_val, val_pred, average='macro')
        scores.append(score)
        print(f"  Fold {i+1}: F1 = {score:.4f}, Threshold = {threshold:.3f}")

    return np.mean(scores), np.std(scores)

print("🤖 Training elite ensemble model...")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Perform temporal cross-validation
print("📊 Performing temporal cross-validation...")
rf_cv_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
cv_score, cv_std = temporal_cross_validation(rf_cv_model, X_train_scaled, y_train, X_train_elite['Week'])
print(f"✅ Cross-validation score: {cv_score:.4f} ± {cv_std:.4f}")

# Train optimized ensemble
print("🎯 Training final ensemble models...")
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight='balanced_subsample',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

gb_model = GradientBoostingClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# Train both models
rf_model.fit(X_train_scaled, y_train)
gb_model.fit(X_train_scaled, y_train)

# Option 1: Simple weighted ensemble
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
gb_proba = gb_model.predict_proba(X_test_scaled)[:, 1]
ensemble_proba = rf_proba * 0.7 + gb_proba * 0.3

# Option 2: Stacked ensemble (uncomment to use)
# print("🔄 Training stacked ensemble...")
# stacked_proba = create_stacked_ensemble(X_train_scaled, y_train, X_test_scaled)
# ensemble_proba = stacked_proba  # Use stacked ensemble instead

print("📊 Analyzing optimal threshold...")

# Find optimal threshold for Macro-F1 with business constraints
optimal_threshold, optimal_f1 = optimize_threshold_with_business_rules(
    y_train,
    rf_model.predict_proba(X_train_scaled)[:, 1]  # Use RF for threshold optimization
)

print(f"✅ Optimal threshold: {optimal_threshold:.3f}")
print(f"   Expected F1 score: {optimal_f1:.4f}")

# Final predictions
final_predictions = (ensemble_proba > optimal_threshold).astype(int)

# Create submission
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': final_predictions
})

submission_df.to_csv("retailbanking_challenge2_elite_predictions.csv", index=False)

print(f"🎉 Elite predictions saved: {len(final_predictions)} predictions")
print(f"   Default rate: {final_predictions.mean():.3f} ({final_predictions.sum()} defaults)")

# Model interpretation
def explain_predictions(model, feature_names, top_n=20):
    """Provide business-interpretable feature importance"""
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(top_n)

    # Categorize features for business interpretation
    categories = {
        'Credit Risk': ['CreditScore', 'credit_utilization', 'high_risk_profile'],
        'Payment Behavior': ['PaymentRatio', 'payment_trend', 'payment_volatility'],
        'Spending Patterns': ['utilization_ma', 'transaction_risk_score', 'spending_volatility'],
        'Session Behavior': ['session_risk_score', 'financial_actions', 'sensitive_action_ratio']
    }

    for feature in importance_df['feature']:
        for category, keywords in categories.items():
            if any(keyword in feature for keyword in keywords):
                importance_df.loc[importance_df['feature'] == feature, 'category'] = category
                break
        else:
            importance_df.loc[importance_df['feature'] == feature, 'category'] = 'Other'

    return importance_df

print("🔍 Top predictive features:")
feature_explanation = explain_predictions(rf_model, top_features)
print(feature_explanation[['feature', 'category', 'importance']].head(10))

# Submit predictions
try:
    from agentds import BenchmarkClient
    client = BenchmarkClient(api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769", team_name="synergy-minds")

    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_elite_predictions.csv")

    if result['success']:
        print("🏆 Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        current_score = result['score']
        if current_score < 0.9:
            improvement_needed = 0.931 - current_score
            print(f"   🎯 Need improvement: {improvement_needed:.4f} to reach top score")
        else:
            print("   💪 Excellent! You're in the top tier!")
    else:
        print("❌ Submission failed!")

except Exception as e:
    print(f"💥 Submission error: {e}")

print("\n💡 Elite strategy features:")
print("   • Advanced temporal dynamics with trend analysis")
print("   • Sophisticated customer segmentation and risk profiling")
print("   • Comprehensive transaction behavior intelligence")
print("   • Session-based security and risk indicators")
print("   • Ensemble modeling with optimized threshold selection")
print("   • Elite feature selection focusing on predictive power")
print("   • Business-constrained threshold optimization")
print("   • Temporal cross-validation for robust performance")
print("   • Feature interaction engineering")
print("   • Model interpretation and business insights")

🚀 Loading and preparing data for elite performance...
🎯 Creating elite feature engineering pipeline...
🔄 Building elite feature sets...
🎯 Creating final feature matrix...
✅ Elite feature matrix: 125 features, 13301 samples
🔍 Performing elite feature selection...
🎯 Selected 50 most predictive features
🤖 Training elite ensemble model...
📊 Performing temporal cross-validation...
  Fold 1: F1 = 0.9420, Threshold = 0.294
  Fold 2: F1 = 0.9432, Threshold = 0.254
  Fold 3: F1 = 0.9465, Threshold = 0.221
  Fold 4: F1 = 0.9476, Threshold = 0.213
  Fold 5: F1 = 0.9379, Threshold = 0.322
✅ Cross-validation score: 0.9434 ± 0.0034
🎯 Training final ensemble models...
📊 Analyzing optimal threshold...
✅ Optimal threshold: 0.452
   Expected F1 score: 0.9989
🎉 Elite predictions saved: 13290 predictions
   Default rate: 0.094 (1249 defaults)
🔍 Top predictive features:
                   feature          category  importance
0      unique_action_count             Other    0.187503
3              CreditSco

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("XGBoost not available, using alternative models")

print("🚀 Loading and preparing data for elite performance...")

# Load data
customers = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customers_all.csv")
accounts = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv")
transactions_train = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_train.csv")
transactions_test = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/transactions_test.csv")
train_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv")
test_panel = pd.read_csv("/home/jovyan/shared/datasets/RetailBanking/customer_panel_test.csv")

with open("/home/jovyan/shared/datasets/RetailBanking/device_sessions_all.json", 'r') as f:
    import json
    device_sessions = pd.json_normalize(json.load(f))

print("🎯 Creating elite feature engineering pipeline...")

def create_elite_customer_features(customers, accounts):
    """Create sophisticated customer segmentation and risk profiling"""
    features = customers.copy()

    # Advanced encoding
    le_city = LabelEncoder()
    features['HomeCity_encoded'] = le_city.fit_transform(features['HomeCity'].fillna('Unknown'))

    # Account portfolio analysis
    account_metrics = accounts.groupby('CustomerID').agg({
        'AccountID': 'count',
        'Balance': ['sum', 'mean', 'std', 'max', 'min', 'median'],
        'Limit': ['sum', 'max', 'mean'],
        'Type': lambda x: x.nunique()
    }).reset_index()
    account_metrics.columns = ['CustomerID', 'total_accounts', 'balance_sum', 'balance_mean',
                              'balance_std', 'balance_max', 'balance_min', 'balance_median',
                              'limit_sum', 'limit_max', 'limit_mean', 'account_type_diversity']

    # Credit-specific features
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_metrics = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['sum', 'mean', 'max'],
            'Limit': ['sum', 'max', 'mean']
        }).reset_index()
        credit_metrics.columns = ['CustomerID', 'credit_balance_sum', 'credit_balance_mean',
                                 'credit_balance_max', 'credit_limit_sum', 'credit_limit_max',
                                 'credit_limit_mean']

        credit_metrics['credit_utilization'] = credit_metrics['credit_balance_sum'] / (credit_metrics['credit_limit_sum'] + 1)
        credit_metrics['max_credit_utilization'] = credit_metrics['credit_balance_max'] / (credit_metrics['credit_limit_max'] + 1)
    else:
        credit_metrics = pd.DataFrame(columns=['CustomerID', 'credit_utilization', 'max_credit_utilization'])

    # Account type composition
    account_composition = pd.get_dummies(accounts[['CustomerID', 'Type']], columns=['Type'], prefix='account')
    account_composition = account_composition.groupby('CustomerID').sum().reset_index()

    # Merge features
    features = features.merge(account_metrics, on='CustomerID', how='left')
    if 'CustomerID' in credit_metrics.columns:
        features = features.merge(credit_metrics, on='CustomerID', how='left')
    features = features.merge(account_composition, on='CustomerID', how='left')

    # Advanced financial ratios
    features['balance_to_salary'] = features['balance_sum'] / (features['AnnualSalary'] + 1)
    features['limit_to_salary'] = features['limit_sum'] / (features['AnnualSalary'] + 1)
    features['credit_depth'] = features['credit_limit_sum'] / (features['AnnualSalary'] + 1)

    # Risk profiling
    features['high_risk_profile'] = (
        (features['CreditScore'] < 580).astype(int) * 0.4 +
        (features.get('credit_utilization', 0) > 0.8).astype(int) * 0.3 +
        (features['balance_to_salary'] > 0.5).astype(int) * 0.3
    )

    # Customer lifetime value proxy
    features['clv_score'] = (
        (features['AnnualSalary'] / features['AnnualSalary'].max()) * 0.4 +
        (features['CreditScore'] / 850) * 0.3 +
        (features['Tenure'] / features['Tenure'].max()) * 0.3
    )

    return features.fillna(0)

def enhance_session_intelligence(device_sessions):
    """Build on your successful session features"""
    sessions = device_sessions.copy()

    # Action sequence analysis - FIXED VERSION
    def analyze_action_sequences(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'action_sequence_complexity': 0,
                'financial_action_velocity': 0,
                'sensitive_action_clustering': 0
            }

            financial_actions = []
            action_types = []

            for i, action in enumerate(actions_list):
                # Handle both string and dictionary actions
                if isinstance(action, dict):
                    action_type = action.get('type', 'unknown')
                    action_types.append(action_type)

                    if action_type in ['transfer', 'payment']:
                        financial_actions.append(i)
                elif isinstance(action, str):
                    action_types.append(action)
                    if action in ['transfer', 'payment']:
                        financial_actions.append(i)

            # Calculate financial action velocity
            if len(financial_actions) > 1:
                stats['financial_action_velocity'] = len(financial_actions) / (financial_actions[-1] - financial_actions[0] + 1)

            # Action sequence complexity (entropy)
            if action_types:
                unique, counts = np.unique(action_types, return_counts=True)
                probs = counts / len(action_types)
                stats['action_sequence_complexity'] = -np.sum(probs * np.log2(probs + 1e-10))

            return stats
        return {'action_sequence_complexity': 0, 'financial_action_velocity': 0, 'sensitive_action_clustering': 0}

    action_sequence_analysis = sessions['Actions'].apply(analyze_action_sequences)
    sequence_df = pd.DataFrame(action_sequence_analysis.tolist())
    sequence_df['CustomerID'] = sessions['CustomerID'].values

    return sequence_df.groupby('CustomerID').mean().reset_index()

def enhance_credit_behavior_features(panel_df, accounts):
    """Leverage your successful credit risk features"""
    features = panel_df.copy()

    # Credit utilization patterns
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_behavior = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['mean', 'std', 'skew'],
            'Limit': ['mean', 'min', 'max']
        }).reset_index()
        credit_behavior.columns = ['CustomerID', 'credit_balance_mean', 'credit_balance_std', 'credit_balance_skew',
                                 'credit_limit_mean', 'credit_limit_min', 'credit_limit_max']

        # Credit limit utilization patterns
        credit_behavior['limit_utilization_variance'] = credit_behavior['credit_balance_std'] / (credit_behavior['credit_limit_mean'] + 1)
        credit_behavior['limit_adequacy'] = credit_behavior['credit_limit_mean'] / (credit_behavior['credit_limit_max'] + 1)

        features = features.merge(credit_behavior, on='CustomerID', how='left')

    # Payment behavior sophistication
    features['payment_consistency'] = 1 - features.groupby('CustomerID')['PaymentRatio'].transform('std').fillna(0)
    features['utilization_stability'] = 1 - features.groupby('CustomerID')['Utilisation'].transform('std').fillna(0)

    return features.fillna(0)

def detect_behavioral_regimes(panel_df):
    """Detect different behavioral regimes in customer patterns"""
    features = panel_df.copy()

    # Regime detection based on utilization patterns
    features['high_utilization_regime'] = (
        features['Utilisation'].rolling(3, min_periods=1).mean() > 0.7
    ).astype(int)

    features['deteriorating_regime'] = (
        (features['Utilisation'].diff(2) > 0.1) &
        (features['PaymentRatio'].diff(2) < -0.1)
    ).astype(int)

    # Regime persistence
    for customer_id in features['CustomerID'].unique():
        cust_mask = features['CustomerID'] == customer_id
        features.loc[cust_mask, 'regime_persistence'] = (
            features.loc[cust_mask, 'high_utilization_regime'].rolling(4, min_periods=1).mean()
        )

    return features

def enhance_temporal_features(panel_df):
    """Add more sophisticated time-series features"""
    features = panel_df.copy()

    # Seasonal decomposition
    features['utilization_seasonal'] = features.groupby('CustomerID')['Utilisation'].transform(
        lambda x: x - x.rolling(4, min_periods=1).mean()
    )

    # Change point detection
    features['utilization_change_point'] = (
        features['Utilisation'].rolling(3).std() > features['Utilisation'].rolling(6).std() * 1.5
    ).astype(int)

    # Momentum indicators
    features['utilization_momentum'] = features['Utilisation'] - features['Utilisation'].shift(2)
    features['payment_momentum'] = features['PaymentRatio'] - features['PaymentRatio'].shift(2)

    return features

def create_temporal_dynamics_features(panel_df):
    """Create sophisticated time-series features with trend analysis"""
    temporal_features = []

    for customer_id in panel_df['CustomerID'].unique():
        cust_data = panel_df[panel_df['CustomerID'] == customer_id].sort_values('Week')

        # Rolling statistics with multiple windows
        for window in [2, 3, 4]:
            cust_data[f'utilization_ma_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).mean()
            cust_data[f'payment_ratio_ma_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).mean()
            cust_data[f'utilization_std_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).std()
            cust_data[f'payment_ratio_std_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).std()

        # Trend analysis
        cust_data['utilization_trend_3'] = cust_data['Utilisation'].diff(periods=2).fillna(0)
        cust_data['payment_trend_3'] = cust_data['PaymentRatio'].diff(periods=2).fillna(0)

        # Acceleration (second derivative)
        cust_data['utilization_acceleration'] = cust_data['utilization_trend_3'].diff().fillna(0)
        cust_data['payment_acceleration'] = cust_data['payment_trend_3'].diff().fillna(0)

        # Volatility measures
        cust_data['utilization_volatility'] = cust_data['Utilisation'].rolling(4, min_periods=1).std()
        cust_data['payment_volatility'] = cust_data['PaymentRatio'].rolling(4, min_periods=1).std()

        # Behavioral patterns
        cust_data['high_utilization_streak'] = (cust_data['Utilisation'] > 0.7).astype(int)
        cust_data['low_payment_streak'] = (cust_data['PaymentRatio'] < 0.2).astype(int)

        # Calculate streaks
        for i in range(1, len(cust_data)):
            if cust_data.iloc[i]['high_utilization_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('high_utilization_streak')] = \
                    cust_data.iloc[i-1]['high_utilization_streak'] + 1
            if cust_data.iloc[i]['low_payment_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('low_payment_streak')] = \
                    cust_data.iloc[i-1]['low_payment_streak'] + 1

        # Deterioration indicators
        cust_data['financial_deterioration'] = (
            (cust_data['utilization_trend_3'] > 0).astype(int) * 0.5 +
            (cust_data['payment_trend_3'] < 0).astype(int) * 0.5
        )

        temporal_features.append(cust_data)

    result = pd.concat(temporal_features, ignore_index=True).fillna(0)

    # Apply enhanced temporal features
    result = enhance_temporal_features(result)

    return result

def create_advanced_transaction_features(transactions_df):
    """Create comprehensive transaction behavior profiling"""
    transactions = transactions_df.copy()
    transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'], format='ISO8601')

    # Basic transaction metrics
    txn_metrics = transactions.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': ['sum', 'mean', 'std', 'max', 'min', 'median', 'skew'],
        'Timestamp': ['min', 'max', 'nunique']
    }).reset_index()
    txn_metrics.columns = ['CustomerID', 'txn_count', 'amount_sum', 'amount_mean', 'amount_std',
                          'amount_max', 'amount_min', 'amount_median', 'amount_skew',
                          'first_txn', 'last_txn', 'active_days']

    # Transaction period and velocity
    txn_metrics['txn_period_days'] = (txn_metrics['last_txn'] - txn_metrics['first_txn']).dt.days + 1
    txn_metrics['daily_txn_frequency'] = txn_metrics['txn_count'] / txn_metrics['txn_period_days']
    txn_metrics['daily_spending'] = txn_metrics['amount_sum'] / txn_metrics['txn_period_days']

    # Large transaction analysis
    large_txn_threshold = transactions['Amount'].quantile(0.8)
    large_txns = transactions[transactions['Amount'] > large_txn_threshold]

    if not large_txns.empty:
        large_txn_stats = large_txns.groupby('CustomerID').agg({
            'TxnID': 'count',
            'Amount': ['mean', 'sum', 'max']
        }).reset_index()
        large_txn_stats.columns = ['CustomerID', 'large_txn_count', 'large_txn_avg',
                                  'large_txn_sum', 'large_txn_max']

        large_txn_stats['large_txn_ratio'] = large_txn_stats['large_txn_count'] / txn_metrics['txn_count']
        large_txn_stats['large_amount_ratio'] = large_txn_stats['large_txn_sum'] / txn_metrics['amount_sum']
    else:
        large_txn_stats = pd.DataFrame(columns=['CustomerID', 'large_txn_ratio', 'large_amount_ratio'])

    # Channel behavior
    channel_behavior = pd.get_dummies(transactions[['CustomerID', 'Channel']],
                                    columns=['Channel'], prefix='channel')
    channel_behavior = channel_behavior.groupby('CustomerID').mean().reset_index()

    # MCC spending patterns
    if 'MCC_Group' in transactions.columns:
        mcc_behavior = pd.get_dummies(transactions[['CustomerID', 'MCC_Group']],
                                    columns=['MCC_Group'], prefix='mcc')
        mcc_behavior = mcc_behavior.groupby('CustomerID').mean().reset_index()
    else:
        mcc_behavior = pd.DataFrame(columns=['CustomerID'])

    # Temporal patterns
    transactions['hour'] = transactions['Timestamp'].dt.hour
    transactions['day_of_week'] = transactions['Timestamp'].dt.dayofweek
    transactions['is_weekend'] = (transactions['day_of_week'] >= 5).astype(int)
    transactions['is_night'] = ((transactions['hour'] >= 22) | (transactions['hour'] <= 6)).astype(int)

    temporal_patterns = transactions.groupby('CustomerID').agg({
        'is_weekend': 'mean',
        'is_night': 'mean',
        'hour': ['mean', 'std', lambda x: x.mode()[0] if len(x.mode()) > 0 else 12]
    }).reset_index()
    temporal_patterns.columns = ['CustomerID', 'weekend_ratio', 'night_ratio',
                                'avg_txn_hour', 'std_txn_hour', 'mode_txn_hour']

    # Spending consistency
    daily_spending = transactions.groupby([transactions['Timestamp'].dt.date, 'CustomerID'])['Amount'].sum().reset_index()
    spending_consistency = daily_spending.groupby('CustomerID')['Amount'].agg(['mean', 'std']).reset_index()
    spending_consistency.columns = ['CustomerID', 'daily_spending_mean', 'daily_spending_std']
    spending_consistency['spending_volatility'] = spending_consistency['daily_spending_std'] / (spending_consistency['daily_spending_mean'] + 1)

    # Merge all features
    features = txn_metrics.merge(channel_behavior, on='CustomerID', how='left')
    if 'CustomerID' in mcc_behavior.columns:
        features = features.merge(mcc_behavior, on='CustomerID', how='left')
    features = features.merge(temporal_patterns, on='CustomerID', how='left')
    features = features.merge(spending_consistency, on='CustomerID', how='left')

    if 'CustomerID' in large_txn_stats.columns:
        features = features.merge(large_txn_stats, on='CustomerID', how='left')

    # Risk scores
    features['transaction_risk_score'] = (
        features.get('large_txn_ratio', 0) * 0.3 +
        features['spending_volatility'] * 0.3 +
        features['night_ratio'] * 0.2 +
        (features['amount_skew'].abs() * 0.2)
    )

    return features.fillna(0)

def create_session_intelligence_features(device_sessions):
    """Create advanced session behavior intelligence"""
    sessions = device_sessions.copy()
    sessions['Timestamp'] = pd.to_datetime(sessions['Timestamp'], format='ISO8601')

    # Session frequency and patterns
    session_freq = sessions.groupby('CustomerID').agg({
        'SessionID': 'count',
        'City': 'nunique',
        'DeviceID': 'nunique',
        'IP': 'nunique',
        'Timestamp': ['min', 'max']
    }).reset_index()
    session_freq.columns = ['CustomerID', 'session_count', 'cities_visited', 'devices_used',
                           'ips_used', 'first_session', 'last_session']

    session_freq['session_period_days'] = (session_freq['last_session'] - session_freq['first_session']).dt.days + 1
    session_freq['daily_sessions'] = session_freq['session_count'] / session_freq['session_period_days']

    # Advanced action analysis - FIXED VERSION
    def analyze_advanced_actions(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'total_actions': len(actions_list),
                'financial_actions': 0,
                'sensitive_actions': 0,
                'login_count': 0,
                'transfer_amount': 0,
                'payment_amount': 0,
                'unique_action_types': set()
            }

            for action in actions_list:
                # Handle both string and dictionary actions
                if isinstance(action, dict):
                    action_type = action.get('type', '')
                    stats['unique_action_types'].add(action_type)

                    if action_type in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                        amount = action.get('amount', 0)
                        if action_type == 'transfer':
                            stats['transfer_amount'] += amount
                        else:
                            stats['payment_amount'] += amount
                    elif action_type == 'account_view':
                        stats['sensitive_actions'] += 1
                elif isinstance(action, str):
                    stats['unique_action_types'].add(action)
                    if action == 'login':
                        stats['login_count'] += 1
                    elif action in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                    elif action == 'account_view':
                        stats['sensitive_actions'] += 1

            stats['unique_action_count'] = len(stats['unique_action_types'])
            return stats
        return {'total_actions': 0, 'financial_actions': 0, 'sensitive_actions': 0,
                'login_count': 0, 'transfer_amount': 0, 'payment_amount': 0, 'unique_action_count': 0}

    action_analysis = sessions['Actions'].apply(analyze_advanced_actions)
    action_df = pd.DataFrame(action_analysis.tolist())
    action_df['CustomerID'] = sessions['CustomerID'].values

    # Aggregate action intelligence
    action_intel = action_df.groupby('CustomerID').agg({
        'total_actions': 'sum',
        'financial_actions': 'sum',
        'sensitive_actions': 'sum',
        'login_count': 'sum',
        'transfer_amount': 'sum',
        'payment_amount': 'sum',
        'unique_action_count': 'mean'
    }).reset_index()

    # Calculate behavioral ratios
    action_intel['financial_action_ratio'] = action_intel['financial_actions'] / (action_intel['total_actions'] + 1)
    action_intel['sensitive_action_ratio'] = action_intel['sensitive_actions'] / (action_intel['total_actions'] + 1)
    action_intel['login_frequency'] = action_intel['login_count'] / session_freq['session_count']
    action_intel['avg_financial_amount'] = (action_intel['transfer_amount'] + action_intel['payment_amount']) / (action_intel['financial_actions'] + 1)

    # Merge features
    features = session_freq.merge(action_intel, on='CustomerID', how='left')

    # Security and risk indicators
    features['geographic_dispersion'] = features['cities_visited'] / (features['session_count'] + 1)
    features['device_diversity'] = features['devices_used'] / (features['session_count'] + 1)
    features['ip_diversity'] = features['ips_used'] / (features['session_count'] + 1)

    features['session_risk_score'] = (
        features['geographic_dispersion'] * 0.3 +
        features['device_diversity'] * 0.3 +
        features['sensitive_action_ratio'] * 0.4
    )

    return features.fillna(0)

def create_interaction_features(features):
    """Create powerful interaction terms"""
    # Financial stress interactions
    if all(col in features.columns for col in ['utilization_ma_3', 'payment_ratio_ma_3']):
        features['utilization_payment_interaction'] = (
            features['utilization_ma_3'] * (1 - features['payment_ratio_ma_3'])
        )

    # Credit-behavior interactions
    if all(col in features.columns for col in ['CreditScore', 'transaction_risk_score']):
        features['credit_behavior_risk'] = (
            (1 - features['CreditScore'] / 850) * features['transaction_risk_score']
        )

    # Multi-dimensional risk scoring
    risk_components = []
    risk_cols = ['high_risk_profile', 'transaction_risk_score', 'session_risk_score', 'financial_deterioration']

    for col in risk_cols:
        if col in features.columns:
            risk_components.append(features[col])

    if risk_components:
        features['comprehensive_risk_index'] = sum(risk_components) / len(risk_components)

    # Session-credit interactions
    if all(col in features.columns for col in ['unique_action_count', 'CreditScore']):
        features['session_credit_interaction'] = (
            features['unique_action_count'] * (1 - features['CreditScore'] / 850)
        )

    return features

print("🔄 Building elite feature sets...")

# Create all feature sets
customer_features = create_elite_customer_features(customers, accounts)
temporal_train = create_temporal_dynamics_features(train_panel)
temporal_test = create_temporal_dynamics_features(test_panel)

# Combine train and test transactions for consistent feature engineering
all_transactions = pd.concat([transactions_train, transactions_test])
transaction_features = create_advanced_transaction_features(all_transactions)
session_features = create_session_intelligence_features(device_sessions)

# Enhanced features
enhanced_session_features = enhance_session_intelligence(device_sessions)

print("🎯 Creating final feature matrix...")

def create_elite_feature_matrix(panel_df, customer_df, temporal_df, transaction_df, session_df, enhanced_session_df):
    """Combine all elite features with intelligent interactions"""
    features = temporal_df.copy()

    # Merge all feature sources
    features = features.merge(customer_df, on='CustomerID', how='left')
    features = features.merge(transaction_df, on='CustomerID', how='left')
    features = features.merge(session_df, on='CustomerID', how='left')
    features = features.merge(enhanced_session_df, on='CustomerID', how='left')

    # Enhanced credit behavior features
    features = enhance_credit_behavior_features(features, accounts)

    # Behavioral regime detection
    features = detect_behavioral_regimes(features)

    # Create powerful interaction features
    # Financial stress indicators
    stress_components = []
    if 'utilization_ma_3' in features.columns:
        stress_components.append(features['utilization_ma_3'] * 0.25)
    if 'payment_ratio_ma_3' in features.columns:
        stress_components.append((1 - features['payment_ratio_ma_3']) * 0.25)
    if 'high_risk_profile' in features.columns:
        stress_components.append(features['high_risk_profile'] * 0.25)
    if 'transaction_risk_score' in features.columns:
        stress_components.append(features['transaction_risk_score'] * 0.25)

    if stress_components:
        features['comprehensive_stress_score'] = sum(stress_components)

    # Behavioral risk indicators
    behavior_components = []
    if 'session_risk_score' in features.columns:
        behavior_components.append(features['session_risk_score'] * 0.4)
    if 'financial_deterioration' in features.columns:
        behavior_components.append(features['financial_deterioration'] * 0.3)
    if 'spending_volatility' in features.columns:
        behavior_components.append(features['spending_volatility'] * 0.3)

    if behavior_components:
        features['behavioral_risk_score'] = sum(behavior_components)

    # Credit capacity indicators
    if 'CreditScore' in features.columns and 'credit_utilization' in features.columns:
        features['credit_health_index'] = (
            (features['CreditScore'] / 850) * 0.6 +
            (1 - features['credit_utilization'].clip(0, 1)) * 0.4
        )

    # Payment behavior indicators
    if 'payment_trend_3' in features.columns and 'payment_volatility' in features.columns:
        features['payment_behavior_score'] = (
            (1 - features['payment_trend_3'].clip(-1, 0).abs()) * 0.5 +
            (1 - features['payment_volatility'].clip(0, 1)) * 0.5
        )

    # Apply interaction features
    features = create_interaction_features(features)

    # Final composite risk score
    risk_components_final = []
    if 'comprehensive_stress_score' in features.columns:
        risk_components_final.append(features['comprehensive_stress_score'] * 0.3)
    if 'behavioral_risk_score' in features.columns:
        risk_components_final.append(features['behavioral_risk_score'] * 0.3)
    if 'comprehensive_risk_index' in features.columns:
        risk_components_final.append(features['comprehensive_risk_index'] * 0.2)
    if 'session_risk_score' in features.columns:
        risk_components_final.append(features['session_risk_score'] * 0.2)

    if risk_components_final:
        features['final_risk_score'] = sum(risk_components_final)

    return features.fillna(0)

# Create final datasets
X_train_elite = create_elite_feature_matrix(train_panel, customer_features, temporal_train, transaction_features, session_features, enhanced_session_features)
X_test_elite = create_elite_feature_matrix(test_panel, customer_features, temporal_test, transaction_features, session_features, enhanced_session_features)

# Prepare for modeling
y_train = X_train_elite['DefaultLabel'].astype(int)
non_feature_cols = ['CustomerID', 'Week', 'DefaultLabel', 'first_txn', 'last_txn', 'first_session', 'last_session']
feature_cols = [col for col in X_train_elite.columns if col not in non_feature_cols]

X_train = X_train_elite[feature_cols]
X_test = X_test_elite[feature_cols]

print(f"✅ Elite feature matrix: {X_train.shape[1]} features, {X_train.shape[0]} samples")

# Ensure numeric types and handle infinite values
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)

print("🔍 Performing elite feature selection...")

# Use Random Forest for feature selection
selector_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
selector_model.fit(X_train, y_train)

# Select top features based on importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': selector_model.feature_importances_
}).sort_values('importance', ascending=False)

# Keep top 60 features for enhanced model
top_features = feature_importance.head(60)['feature'].tolist()
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(f"🎯 Selected {len(top_features)} most predictive features")

def optimize_threshold_with_business_rules(y_true, y_proba, min_default_rate=0.05, max_default_rate=0.15):
    """Optimize threshold considering business constraints"""
    thresholds = np.linspace(0.1, 0.5, 100)
    best_threshold = 0.25
    best_f1 = 0

    for threshold in thresholds:
        y_pred = (y_proba > threshold).astype(int)
        default_rate = y_pred.mean()

        # Apply business constraints
        if min_default_rate <= default_rate <= max_default_rate:
            f1 = f1_score(y_true, y_pred, average='macro')

            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold

    return best_threshold, best_f1

def dynamic_ensemble_weighting(rf_proba, gb_proba, customer_features):
    """Dynamic ensemble weighting based on customer characteristics"""
    # Default weights
    default_rf_weight = 0.7

    # If we have session complexity features, adjust weights
    if 'action_sequence_complexity' in customer_features.columns:
        session_complexity = customer_features['action_sequence_complexity'].fillna(0)
        # Higher RF weight for complex session behavior
        rf_weights = np.where(
            session_complexity > session_complexity.quantile(0.7),
            0.8,  # More RF for complex sessions
            default_rf_weight
        )
    else:
        rf_weights = np.full(len(rf_proba), default_rf_weight)

    # If we have credit scores, adjust weights
    if 'CreditScore' in customer_features.columns:
        credit_scores = customer_features['CreditScore'].fillna(650)
        # Higher GB weight for strong credit profiles
        credit_adjustment = np.where(
            credit_scores > 700,
            -0.1,  # Less RF for good credit
            0.1    # More RF for poor credit
        )
        rf_weights = np.clip(rf_weights + credit_adjustment, 0.5, 0.9)

    # Apply dynamic weighting
    ensemble_proba = np.zeros_like(rf_proba)
    for i in range(len(ensemble_proba)):
        ensemble_proba[i] = (rf_proba[i] * rf_weights[i] +
                           gb_proba[i] * (1 - rf_weights[i]))

    return ensemble_proba

def create_stacked_ensemble(X_train, y_train, X_test):
    """Create stacked ensemble with multiple base models"""
    base_models = {
        'rf': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
        'gb': GradientBoostingClassifier(n_estimators=150, random_state=42)
    }

    # Add XGBoost if available
    if XGB_AVAILABLE:
        base_models['xgb'] = xgb.XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    # Create meta-features using cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    meta_train = np.zeros((X_train.shape[0], len(base_models)))
    meta_test = np.zeros((X_test.shape[0], len(base_models)))

    for i, (name, model) in enumerate(base_models.items()):
        fold_predictions = np.zeros(X_train.shape[0])
        test_predictions = []

        for train_idx, val_idx in skf.split(X_train, y_train):
            model.fit(X_train[train_idx], y_train[train_idx])
            fold_predictions[val_idx] = model.predict_proba(X_train[val_idx])[:, 1]
            test_predictions.append(model.predict_proba(X_test)[:, 1])

        meta_train[:, i] = fold_predictions
        meta_test[:, i] = np.mean(test_predictions, axis=0)

    # Meta-model
    meta_model = LogisticRegression()
    meta_model.fit(meta_train, y_train)

    return meta_model.predict_proba(meta_test)[:, 1]

def temporal_cross_validation(model, X, y, weeks, n_splits=5):
    """Time-aware cross-validation"""
    unique_weeks = sorted(weeks.unique())
    split_size = len(unique_weeks) // n_splits

    scores = []
    thresholds = []

    for i in range(n_splits):
        val_weeks = unique_weeks[i * split_size:(i + 1) * split_size]

        train_mask = ~weeks.isin(val_weeks)
        val_mask = weeks.isin(val_weeks)

        X_train_fold, X_val = X[train_mask], X[val_mask]
        y_train_fold, y_val = y[train_mask], y[val_mask]

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict_proba(X_val)[:, 1]

        # Optimize threshold for this fold
        threshold, _ = optimize_threshold_with_business_rules(y_val, y_pred)
        val_pred = (y_pred > threshold).astype(int)

        score = f1_score(y_val, val_pred, average='macro')
        scores.append(score)
        thresholds.append(threshold)
        print(f"  Fold {i+1}: F1 = {score:.4f}, Threshold = {threshold:.3f}")

    return np.mean(scores), np.std(scores), np.mean(thresholds)

print("🤖 Training elite ensemble model...")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Perform temporal cross-validation
print("📊 Performing temporal cross-validation...")
rf_cv_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
cv_score, cv_std, avg_threshold = temporal_cross_validation(rf_cv_model, X_train_scaled, y_train, X_train_elite['Week'])
print(f"✅ Cross-validation score: {cv_score:.4f} ± {cv_std:.4f}")
print(f"✅ Average optimal threshold: {avg_threshold:.3f}")

# Train optimized ensemble
print("🎯 Training final ensemble models...")
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight='balanced_subsample',
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

gb_model = GradientBoostingClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

# Train both models
rf_model.fit(X_train_scaled, y_train)
gb_model.fit(X_train_scaled, y_train)

# Get base predictions
rf_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
gb_proba = gb_model.predict_proba(X_test_scaled)[:, 1]

# Use dynamic ensemble weighting
ensemble_proba = dynamic_ensemble_weighting(rf_proba, gb_proba, X_test_elite)

print("📊 Analyzing optimal threshold...")

# Find optimal threshold for Macro-F1 with business constraints
optimal_threshold, optimal_f1 = optimize_threshold_with_business_rules(
    y_train,
    rf_model.predict_proba(X_train_scaled)[:, 1]  # Use RF for threshold optimization
)

print(f"✅ Optimal threshold: {optimal_threshold:.3f}")
print(f"   Expected F1 score: {optimal_f1:.4f}")

# Final predictions
final_predictions = (ensemble_proba > optimal_threshold).astype(int)

# Create submission
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': final_predictions
})

submission_df.to_csv("retailbanking_challenge2_elite_enhanced_predictions.csv", index=False)

print(f"🎉 Elite enhanced predictions saved: {len(final_predictions)} predictions")
print(f"   Default rate: {final_predictions.mean():.3f} ({final_predictions.sum()} defaults)")

# Model interpretation
def explain_predictions(model, feature_names, top_n=20):
    """Provide business-interpretable feature importance"""
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(top_n)

    # Categorize features for business interpretation
    categories = {
        'Credit Risk': ['CreditScore', 'credit_utilization', 'high_risk_profile', 'credit_health'],
        'Payment Behavior': ['PaymentRatio', 'payment_trend', 'payment_volatility', 'payment_consistency'],
        'Spending Patterns': ['utilization_ma', 'transaction_risk_score', 'spending_volatility'],
        'Session Behavior': ['session_risk_score', 'financial_actions', 'sensitive_action_ratio', 'action_sequence'],
        'Behavioral Regimes': ['regime', 'deteriorating', 'persistence'],
        'Composite Scores': ['comprehensive_risk', 'final_risk', 'stress_score']
    }

    for feature in importance_df['feature']:
        category_found = False
        for category, keywords in categories.items():
            if any(keyword in feature for keyword in keywords):
                importance_df.loc[importance_df['feature'] == feature, 'category'] = category
                category_found = True
                break
        if not category_found:
            importance_df.loc[importance_df['feature'] == feature, 'category'] = 'Other'

    return importance_df

print("🔍 Top predictive features:")
feature_explanation = explain_predictions(rf_model, top_features)
print(feature_explanation[['feature', 'category', 'importance']].head(15))

# Submit predictions
try:
    from agentds import BenchmarkClient
    client = BenchmarkClient(api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769", team_name="synergy-minds")

    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_elite_enhanced_predictions.csv")

    if result['success']:
        print("🏆 Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        current_score = result['score']
        if current_score < 0.95:
            improvement_needed = 0.95 - current_score
            print(f"   🎯 Need improvement: {improvement_needed:.4f} to reach elite tier")
        else:
            print("   💪 Excellent! You're in the elite tier!")

        # Compare with previous score
        if current_score > 0.9438:
            improvement = current_score - 0.9438
            print(f"   📈 Improvement from previous: +{improvement:.4f}")
    else:
        print("❌ Submission failed!")

except Exception as e:
    print(f"💥 Submission error: {e}")

print("\n💡 Elite Enhanced Strategy Features:")
print("   • Advanced session sequence analysis with complexity scoring")
print("   • Enhanced credit behavior patterns and limit utilization")
print("   • Behavioral regime detection with persistence tracking")
print("   • Dynamic ensemble weighting based on customer profiles")
print("   • Comprehensive interaction feature engineering")
print("   • Multi-dimensional risk scoring integration")
print("   • Temporal cross-validation with business constraints")
print("   • Advanced feature selection (60 top features)")
print("   • Session-credit interaction modeling")
print("   • Real-time threshold optimization")

🚀 Loading and preparing data for elite performance...
🎯 Creating elite feature engineering pipeline...
🔄 Building elite feature sets...
🎯 Creating final feature matrix...
✅ Elite feature matrix: 143 features, 13301 samples
🔍 Performing elite feature selection...
🎯 Selected 60 most predictive features
🤖 Training elite ensemble model...
📊 Performing temporal cross-validation...
  Fold 1: F1 = 0.9425, Threshold = 0.294
  Fold 2: F1 = 0.9442, Threshold = 0.254
  Fold 3: F1 = 0.9455, Threshold = 0.241
  Fold 4: F1 = 0.9478, Threshold = 0.193
  Fold 5: F1 = 0.9388, Threshold = 0.330
✅ Cross-validation score: 0.9438 ± 0.0030
✅ Average optimal threshold: 0.262
🎯 Training final ensemble models...
📊 Analyzing optimal threshold...
✅ Optimal threshold: 0.431
   Expected F1 score: 1.0000
🎉 Elite enhanced predictions saved: 13290 predictions
   Default rate: 0.099 (1311 defaults)
🔍 Top predictive features:
                       feature          category  importance
0   action_sequence_complexity  S

In [ ]:
#  Version for 0.9553
import pandas as pd
import numpy as np
import random
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

# Set all random seeds for maximum reproducibility
def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    try:
        import torch
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

set_all_seeds(42)

print("🚀 Loading and preparing data with enhanced stability...")

# Load data
customers = pd.read_csv("customers_all.csv")
accounts = pd.read_csv("accounts_all.csv")
transactions_train = pd.read_csv("transactions_train.csv")
transactions_test = pd.read_csv("transactions_test.csv")
train_panel = pd.read_csv("customer_panel_train.csv")
test_panel = pd.read_csv("customer_panel_test.csv")

with open("device_sessions_all.json", 'r') as f:
    import json
    device_sessions = pd.json_normalize(json.load(f))

# STABLE FEATURE ENGINEERING FUNCTIONS (keep your existing functions but add stability)

def create_elite_customer_features(customers, accounts):
    """Stable customer feature engineering"""
    features = customers.copy()

    # Stable encoding
    le_city = LabelEncoder()
    features['HomeCity_encoded'] = le_city.fit_transform(features['HomeCity'].fillna('Unknown'))

    # Account portfolio analysis (same as before)
    account_metrics = accounts.groupby('CustomerID').agg({
        'AccountID': 'count',
        'Balance': ['sum', 'mean', 'std', 'max', 'min', 'median'],
        'Limit': ['sum', 'max', 'mean'],
        'Type': lambda x: x.nunique()
    }).reset_index()
    account_metrics.columns = ['CustomerID', 'total_accounts', 'balance_sum', 'balance_mean',
                              'balance_std', 'balance_max', 'balance_min', 'balance_median',
                              'limit_sum', 'limit_max', 'limit_mean', 'account_type_diversity']

    # Credit-specific features
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_metrics = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['sum', 'mean', 'max'],
            'Limit': ['sum', 'max', 'mean']
        }).reset_index()
        credit_metrics.columns = ['CustomerID', 'credit_balance_sum', 'credit_balance_mean',
                                 'credit_balance_max', 'credit_limit_sum', 'credit_limit_max',
                                 'credit_limit_mean']
        credit_metrics['credit_utilization'] = credit_metrics['credit_balance_sum'] / (credit_metrics['credit_limit_sum'] + 1)
        credit_metrics['max_credit_utilization'] = credit_metrics['credit_balance_max'] / (credit_metrics['credit_limit_max'] + 1)
    else:
        credit_metrics = pd.DataFrame(columns=['CustomerID', 'credit_utilization', 'max_credit_utilization'])

    # Account type composition
    account_composition = pd.get_dummies(accounts[['CustomerID', 'Type']], columns=['Type'], prefix='account')
    account_composition = account_composition.groupby('CustomerID').sum().reset_index()

    # Merge features
    features = features.merge(account_metrics, on='CustomerID', how='left')
    if 'CustomerID' in credit_metrics.columns:
        features = features.merge(credit_metrics, on='CustomerID', how='left')
    features = features.merge(account_composition, on='CustomerID', how='left')

    # Advanced financial ratios
    features['balance_to_salary'] = features['balance_sum'] / (features['AnnualSalary'] + 1)
    features['limit_to_salary'] = features['limit_sum'] / (features['AnnualSalary'] + 1)
    features['credit_depth'] = features['credit_limit_sum'] / (features['AnnualSalary'] + 1)

    # Risk profiling
    features['high_risk_profile'] = (
        (features['CreditScore'] < 580).astype(int) * 0.4 +
        (features.get('credit_utilization', 0) > 0.8).astype(int) * 0.3 +
        (features['balance_to_salary'] > 0.5).astype(int) * 0.3
    )

    # Customer lifetime value proxy
    features['clv_score'] = (
        (features['AnnualSalary'] / features['AnnualSalary'].max()) * 0.4 +
        (features['CreditScore'] / 850) * 0.3 +
        (features['Tenure'] / features['Tenure'].max()) * 0.3
    )

    return features.fillna(0)

# KEEP YOUR OTHER FEATURE ENGINEERING FUNCTIONS BUT ADD STABILITY
# (enhance_session_intelligence, enhance_credit_behavior_features, detect_behavioral_regimes,
# enhance_temporal_features, create_temporal_dynamics_features, create_advanced_transaction_features,
# create_session_intelligence_features, create_interaction_features, create_elite_feature_matrix)


def enhance_session_intelligence(device_sessions):
    """Build on your successful session features"""
    sessions = device_sessions.copy()

    # Action sequence analysis - FIXED VERSION
    def analyze_action_sequences(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'action_sequence_complexity': 0,
                'financial_action_velocity': 0,
                'sensitive_action_clustering': 0
            }

            financial_actions = []
            action_types = []

            for i, action in enumerate(actions_list):
                # Handle both string and dictionary actions
                if isinstance(action, dict):
                    action_type = action.get('type', 'unknown')
                    action_types.append(action_type)

                    if action_type in ['transfer', 'payment']:
                        financial_actions.append(i)
                elif isinstance(action, str):
                    action_types.append(action)
                    if action in ['transfer', 'payment']:
                        financial_actions.append(i)

            # Calculate financial action velocity
            if len(financial_actions) > 1:
                stats['financial_action_velocity'] = len(financial_actions) / (financial_actions[-1] - financial_actions[0] + 1)

            # Action sequence complexity (entropy)
            if action_types:
                unique, counts = np.unique(action_types, return_counts=True)
                probs = counts / len(action_types)
                stats['action_sequence_complexity'] = -np.sum(probs * np.log2(probs + 1e-10))

            return stats
        return {'action_sequence_complexity': 0, 'financial_action_velocity': 0, 'sensitive_action_clustering': 0}

    action_sequence_analysis = sessions['Actions'].apply(analyze_action_sequences)
    sequence_df = pd.DataFrame(action_sequence_analysis.tolist())
    sequence_df['CustomerID'] = sessions['CustomerID'].values

    return sequence_df.groupby('CustomerID').mean().reset_index()

def enhance_credit_behavior_features(panel_df, accounts):
    """Leverage your successful credit risk features"""
    features = panel_df.copy()

    # Credit utilization patterns
    credit_accounts = accounts[accounts['Type'] == 'credit_card']
    if not credit_accounts.empty:
        credit_behavior = credit_accounts.groupby('CustomerID').agg({
            'Balance': ['mean', 'std', 'skew'],
            'Limit': ['mean', 'min', 'max']
        }).reset_index()
        credit_behavior.columns = ['CustomerID', 'credit_balance_mean', 'credit_balance_std', 'credit_balance_skew',
                                 'credit_limit_mean', 'credit_limit_min', 'credit_limit_max']

        # Credit limit utilization patterns
        credit_behavior['limit_utilization_variance'] = credit_behavior['credit_balance_std'] / (credit_behavior['credit_limit_mean'] + 1)
        credit_behavior['limit_adequacy'] = credit_behavior['credit_limit_mean'] / (credit_behavior['credit_limit_max'] + 1)

        features = features.merge(credit_behavior, on='CustomerID', how='left')

    # Payment behavior sophistication
    features['payment_consistency'] = 1 - features.groupby('CustomerID')['PaymentRatio'].transform('std').fillna(0)
    features['utilization_stability'] = 1 - features.groupby('CustomerID')['Utilisation'].transform('std').fillna(0)

    return features.fillna(0)

def detect_behavioral_regimes(panel_df):
    """Detect different behavioral regimes in customer patterns"""
    features = panel_df.copy()

    # Regime detection based on utilization patterns
    features['high_utilization_regime'] = (
        features['Utilisation'].rolling(3, min_periods=1).mean() > 0.7
    ).astype(int)

    features['deteriorating_regime'] = (
        (features['Utilisation'].diff(2) > 0.1) &
        (features['PaymentRatio'].diff(2) < -0.1)
    ).astype(int)

    # Regime persistence
    for customer_id in features['CustomerID'].unique():
        cust_mask = features['CustomerID'] == customer_id
        features.loc[cust_mask, 'regime_persistence'] = (
            features.loc[cust_mask, 'high_utilization_regime'].rolling(4, min_periods=1).mean()
        )

    return features

def enhance_temporal_features(panel_df):
    """Add more sophisticated time-series features"""
    features = panel_df.copy()

    # Seasonal decomposition
    features['utilization_seasonal'] = features.groupby('CustomerID')['Utilisation'].transform(
        lambda x: x - x.rolling(4, min_periods=1).mean()
    )

    # Change point detection
    features['utilization_change_point'] = (
        features['Utilisation'].rolling(3).std() > features['Utilisation'].rolling(6).std() * 1.5
    ).astype(int)

    # Momentum indicators
    features['utilization_momentum'] = features['Utilisation'] - features['Utilisation'].shift(2)
    features['payment_momentum'] = features['PaymentRatio'] - features['PaymentRatio'].shift(2)

    return features

def create_temporal_dynamics_features(panel_df):
    """Create sophisticated time-series features with trend analysis"""
    temporal_features = []

    for customer_id in panel_df['CustomerID'].unique():
        cust_data = panel_df[panel_df['CustomerID'] == customer_id].sort_values('Week')

        # Rolling statistics with multiple windows
        for window in [2, 3, 4]:
            cust_data[f'utilization_ma_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).mean()
            cust_data[f'payment_ratio_ma_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).mean()
            cust_data[f'utilization_std_{window}'] = cust_data['Utilisation'].rolling(window, min_periods=1).std()
            cust_data[f'payment_ratio_std_{window}'] = cust_data['PaymentRatio'].rolling(window, min_periods=1).std()

        # Trend analysis
        cust_data['utilization_trend_3'] = cust_data['Utilisation'].diff(periods=2).fillna(0)
        cust_data['payment_trend_3'] = cust_data['PaymentRatio'].diff(periods=2).fillna(0)

        # Acceleration (second derivative)
        cust_data['utilization_acceleration'] = cust_data['utilization_trend_3'].diff().fillna(0)
        cust_data['payment_acceleration'] = cust_data['payment_trend_3'].diff().fillna(0)

        # Volatility measures
        cust_data['utilization_volatility'] = cust_data['Utilisation'].rolling(4, min_periods=1).std()
        cust_data['payment_volatility'] = cust_data['PaymentRatio'].rolling(4, min_periods=1).std()

        # Behavioral patterns
        cust_data['high_utilization_streak'] = (cust_data['Utilisation'] > 0.7).astype(int)
        cust_data['low_payment_streak'] = (cust_data['PaymentRatio'] < 0.2).astype(int)

        # Calculate streaks
        for i in range(1, len(cust_data)):
            if cust_data.iloc[i]['high_utilization_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('high_utilization_streak')] = \
                    cust_data.iloc[i-1]['high_utilization_streak'] + 1
            if cust_data.iloc[i]['low_payment_streak'] == 1:
                cust_data.iloc[i, cust_data.columns.get_loc('low_payment_streak')] = \
                    cust_data.iloc[i-1]['low_payment_streak'] + 1

        # Deterioration indicators
        cust_data['financial_deterioration'] = (
            (cust_data['utilization_trend_3'] > 0).astype(int) * 0.5 +
            (cust_data['payment_trend_3'] < 0).astype(int) * 0.5
        )

        temporal_features.append(cust_data)

    result = pd.concat(temporal_features, ignore_index=True).fillna(0)

    # Apply enhanced temporal features
    result = enhance_temporal_features(result)

    return result

def create_advanced_transaction_features(transactions_df):
    """Create comprehensive transaction behavior profiling"""
    transactions = transactions_df.copy()
    transactions['Timestamp'] = pd.to_datetime(transactions['Timestamp'], format='ISO8601')

    # Basic transaction metrics
    txn_metrics = transactions.groupby('CustomerID').agg({
        'TxnID': 'count',
        'Amount': ['sum', 'mean', 'std', 'max', 'min', 'median', 'skew'],
        'Timestamp': ['min', 'max', 'nunique']
    }).reset_index()
    txn_metrics.columns = ['CustomerID', 'txn_count', 'amount_sum', 'amount_mean', 'amount_std',
                          'amount_max', 'amount_min', 'amount_median', 'amount_skew',
                          'first_txn', 'last_txn', 'active_days']

    # Transaction period and velocity
    txn_metrics['txn_period_days'] = (txn_metrics['last_txn'] - txn_metrics['first_txn']).dt.days + 1
    txn_metrics['daily_txn_frequency'] = txn_metrics['txn_count'] / txn_metrics['txn_period_days']
    txn_metrics['daily_spending'] = txn_metrics['amount_sum'] / txn_metrics['txn_period_days']

    # Large transaction analysis
    large_txn_threshold = transactions['Amount'].quantile(0.8)
    large_txns = transactions[transactions['Amount'] > large_txn_threshold]

    if not large_txns.empty:
        large_txn_stats = large_txns.groupby('CustomerID').agg({
            'TxnID': 'count',
            'Amount': ['mean', 'sum', 'max']
        }).reset_index()
        large_txn_stats.columns = ['CustomerID', 'large_txn_count', 'large_txn_avg',
                                  'large_txn_sum', 'large_txn_max']

        large_txn_stats['large_txn_ratio'] = large_txn_stats['large_txn_count'] / txn_metrics['txn_count']
        large_txn_stats['large_amount_ratio'] = large_txn_stats['large_txn_sum'] / txn_metrics['amount_sum']
    else:
        large_txn_stats = pd.DataFrame(columns=['CustomerID', 'large_txn_ratio', 'large_amount_ratio'])

    # Channel behavior
    channel_behavior = pd.get_dummies(transactions[['CustomerID', 'Channel']],
                                    columns=['Channel'], prefix='channel')
    channel_behavior = channel_behavior.groupby('CustomerID').mean().reset_index()

    # MCC spending patterns
    if 'MCC_Group' in transactions.columns:
        mcc_behavior = pd.get_dummies(transactions[['CustomerID', 'MCC_Group']],
                                    columns=['MCC_Group'], prefix='mcc')
        mcc_behavior = mcc_behavior.groupby('CustomerID').mean().reset_index()
    else:
        mcc_behavior = pd.DataFrame(columns=['CustomerID'])

    # Temporal patterns
    transactions['hour'] = transactions['Timestamp'].dt.hour
    transactions['day_of_week'] = transactions['Timestamp'].dt.dayofweek
    transactions['is_weekend'] = (transactions['day_of_week'] >= 5).astype(int)
    transactions['is_night'] = ((transactions['hour'] >= 22) | (transactions['hour'] <= 6)).astype(int)

    temporal_patterns = transactions.groupby('CustomerID').agg({
        'is_weekend': 'mean',
        'is_night': 'mean',
        'hour': ['mean', 'std', lambda x: x.mode()[0] if len(x.mode()) > 0 else 12]
    }).reset_index()
    temporal_patterns.columns = ['CustomerID', 'weekend_ratio', 'night_ratio',
                                'avg_txn_hour', 'std_txn_hour', 'mode_txn_hour']

    # Spending consistency
    daily_spending = transactions.groupby([transactions['Timestamp'].dt.date, 'CustomerID'])['Amount'].sum().reset_index()
    spending_consistency = daily_spending.groupby('CustomerID')['Amount'].agg(['mean', 'std']).reset_index()
    spending_consistency.columns = ['CustomerID', 'daily_spending_mean', 'daily_spending_std']
    spending_consistency['spending_volatility'] = spending_consistency['daily_spending_std'] / (spending_consistency['daily_spending_mean'] + 1)

    # Merge all features
    features = txn_metrics.merge(channel_behavior, on='CustomerID', how='left')
    if 'CustomerID' in mcc_behavior.columns:
        features = features.merge(mcc_behavior, on='CustomerID', how='left')
    features = features.merge(temporal_patterns, on='CustomerID', how='left')
    features = features.merge(spending_consistency, on='CustomerID', how='left')

    if 'CustomerID' in large_txn_stats.columns:
        features = features.merge(large_txn_stats, on='CustomerID', how='left')

    # Risk scores
    features['transaction_risk_score'] = (
        features.get('large_txn_ratio', 0) * 0.3 +
        features['spending_volatility'] * 0.3 +
        features['night_ratio'] * 0.2 +
        (features['amount_skew'].abs() * 0.2)
    )

    return features.fillna(0)

def create_session_intelligence_features(device_sessions):
    """Create advanced session behavior intelligence"""
    sessions = device_sessions.copy()
    sessions['Timestamp'] = pd.to_datetime(sessions['Timestamp'], format='ISO8601')

    # Session frequency and patterns
    session_freq = sessions.groupby('CustomerID').agg({
        'SessionID': 'count',
        'City': 'nunique',
        'DeviceID': 'nunique',
        'IP': 'nunique',
        'Timestamp': ['min', 'max']
    }).reset_index()
    session_freq.columns = ['CustomerID', 'session_count', 'cities_visited', 'devices_used',
                           'ips_used', 'first_session', 'last_session']

    session_freq['session_period_days'] = (session_freq['last_session'] - session_freq['first_session']).dt.days + 1
    session_freq['daily_sessions'] = session_freq['session_count'] / session_freq['session_period_days']

    # Advanced action analysis - FIXED VERSION
    def analyze_advanced_actions(actions_list):
        if isinstance(actions_list, list):
            stats = {
                'total_actions': len(actions_list),
                'financial_actions': 0,
                'sensitive_actions': 0,
                'login_count': 0,
                'transfer_amount': 0,
                'payment_amount': 0,
                'unique_action_types': set()
            }

            for action in actions_list:
                # Handle both string and dictionary actions
                if isinstance(action, dict):
                    action_type = action.get('type', '')
                    stats['unique_action_types'].add(action_type)

                    if action_type in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                        amount = action.get('amount', 0)
                        if action_type == 'transfer':
                            stats['transfer_amount'] += amount
                        else:
                            stats['payment_amount'] += amount
                    elif action_type == 'account_view':
                        stats['sensitive_actions'] += 1
                elif isinstance(action, str):
                    stats['unique_action_types'].add(action)
                    if action == 'login':
                        stats['login_count'] += 1
                    elif action in ['transfer', 'payment']:
                        stats['financial_actions'] += 1
                        stats['sensitive_actions'] += 1
                    elif action == 'account_view':
                        stats['sensitive_actions'] += 1

            stats['unique_action_count'] = len(stats['unique_action_types'])
            return stats
        return {'total_actions': 0, 'financial_actions': 0, 'sensitive_actions': 0,
                'login_count': 0, 'transfer_amount': 0, 'payment_amount': 0, 'unique_action_count': 0}

    action_analysis = sessions['Actions'].apply(analyze_advanced_actions)
    action_df = pd.DataFrame(action_analysis.tolist())
    action_df['CustomerID'] = sessions['CustomerID'].values

    # Aggregate action intelligence
    action_intel = action_df.groupby('CustomerID').agg({
        'total_actions': 'sum',
        'financial_actions': 'sum',
        'sensitive_actions': 'sum',
        'login_count': 'sum',
        'transfer_amount': 'sum',
        'payment_amount': 'sum',
        'unique_action_count': 'mean'
    }).reset_index()

    # Calculate behavioral ratios
    action_intel['financial_action_ratio'] = action_intel['financial_actions'] / (action_intel['total_actions'] + 1)
    action_intel['sensitive_action_ratio'] = action_intel['sensitive_actions'] / (action_intel['total_actions'] + 1)
    action_intel['login_frequency'] = action_intel['login_count'] / session_freq['session_count']
    action_intel['avg_financial_amount'] = (action_intel['transfer_amount'] + action_intel['payment_amount']) / (action_intel['financial_actions'] + 1)

    # Merge features
    features = session_freq.merge(action_intel, on='CustomerID', how='left')

    # Security and risk indicators
    features['geographic_dispersion'] = features['cities_visited'] / (features['session_count'] + 1)
    features['device_diversity'] = features['devices_used'] / (features['session_count'] + 1)
    features['ip_diversity'] = features['ips_used'] / (features['session_count'] + 1)

    features['session_risk_score'] = (
        features['geographic_dispersion'] * 0.3 +
        features['device_diversity'] * 0.3 +
        features['sensitive_action_ratio'] * 0.4
    )

    return features.fillna(0)

def create_interaction_features(features):
    """Create powerful interaction terms"""
    # Financial stress interactions
    if all(col in features.columns for col in ['utilization_ma_3', 'payment_ratio_ma_3']):
        features['utilization_payment_interaction'] = (
            features['utilization_ma_3'] * (1 - features['payment_ratio_ma_3'])
        )

    # Credit-behavior interactions
    if all(col in features.columns for col in ['CreditScore', 'transaction_risk_score']):
        features['credit_behavior_risk'] = (
            (1 - features['CreditScore'] / 850) * features['transaction_risk_score']
        )

    # Multi-dimensional risk scoring
    risk_components = []
    risk_cols = ['high_risk_profile', 'transaction_risk_score', 'session_risk_score', 'financial_deterioration']

    for col in risk_cols:
        if col in features.columns:
            risk_components.append(features[col])

    if risk_components:
        features['comprehensive_risk_index'] = sum(risk_components) / len(risk_components)

    # Session-credit interactions
    if all(col in features.columns for col in ['unique_action_count', 'CreditScore']):
        features['session_credit_interaction'] = (
            features['unique_action_count'] * (1 - features['CreditScore'] / 850)
        )

    return features

def create_elite_feature_matrix(panel_df, customer_df, temporal_df, transaction_df, session_df, enhanced_session_df):
    """Combine all elite features with intelligent interactions"""
    features = temporal_df.copy()

    # Merge all feature sources
    features = features.merge(customer_df, on='CustomerID', how='left')
    features = features.merge(transaction_df, on='CustomerID', how='left')
    features = features.merge(session_df, on='CustomerID', how='left')
    features = features.merge(enhanced_session_df, on='CustomerID', how='left')

    # Enhanced credit behavior features
    features = enhance_credit_behavior_features(features, accounts)

    # Behavioral regime detection
    features = detect_behavioral_regimes(features)

    # Create powerful interaction features
    # Financial stress indicators
    stress_components = []
    if 'utilization_ma_3' in features.columns:
        stress_components.append(features['utilization_ma_3'] * 0.25)
    if 'payment_ratio_ma_3' in features.columns:
        stress_components.append((1 - features['payment_ratio_ma_3']) * 0.25)
    if 'high_risk_profile' in features.columns:
        stress_components.append(features['high_risk_profile'] * 0.25)
    if 'transaction_risk_score' in features.columns:
        stress_components.append(features['transaction_risk_score'] * 0.25)

    if stress_components:
        features['comprehensive_stress_score'] = sum(stress_components)

    # Behavioral risk indicators
    behavior_components = []
    if 'session_risk_score' in features.columns:
        behavior_components.append(features['session_risk_score'] * 0.4)
    if 'financial_deterioration' in features.columns:
        behavior_components.append(features['financial_deterioration'] * 0.3)
    if 'spending_volatility' in features.columns:
        behavior_components.append(features['spending_volatility'] * 0.3)

    if behavior_components:
        features['behavioral_risk_score'] = sum(behavior_components)

    # Credit capacity indicators
    if 'CreditScore' in features.columns and 'credit_utilization' in features.columns:
        features['credit_health_index'] = (
            (features['CreditScore'] / 850) * 0.6 +
            (1 - features['credit_utilization'].clip(0, 1)) * 0.4
        )

    # Payment behavior indicators
    if 'payment_trend_3' in features.columns and 'payment_volatility' in features.columns:
        features['payment_behavior_score'] = (
            (1 - features['payment_trend_3'].clip(-1, 0).abs()) * 0.5 +
            (1 - features['payment_volatility'].clip(0, 1)) * 0.5
        )

    # Apply interaction features
    features = create_interaction_features(features)

    # Final composite risk score
    risk_components_final = []
    if 'comprehensive_stress_score' in features.columns:
        risk_components_final.append(features['comprehensive_stress_score'] * 0.3)
    if 'behavioral_risk_score' in features.columns:
        risk_components_final.append(features['behavioral_risk_score'] * 0.3)
    if 'comprehensive_risk_index' in features.columns:
        risk_components_final.append(features['comprehensive_risk_index'] * 0.2)
    if 'session_risk_score' in features.columns:
        risk_components_final.append(features['session_risk_score'] * 0.2)

    if risk_components_final:
        features['final_risk_score'] = sum(risk_components_final)

    return features.fillna(0)

print("🔄 Building stable feature sets...")

# Create all feature sets (same as before)
customer_features = create_elite_customer_features(customers, accounts)
temporal_train = create_temporal_dynamics_features(train_panel)
temporal_test = create_temporal_dynamics_features(test_panel)

# Combine train and test transactions for consistent feature engineering
all_transactions = pd.concat([transactions_train, transactions_test])
transaction_features = create_advanced_transaction_features(all_transactions)
session_features = create_session_intelligence_features(device_sessions)
enhanced_session_features = enhance_session_intelligence(device_sessions)

print("🎯 Creating final feature matrix...")

# Create final datasets (same as before)
X_train_elite = create_elite_feature_matrix(train_panel, customer_features, temporal_train, transaction_features, session_features, enhanced_session_features)
X_test_elite = create_elite_feature_matrix(test_panel, customer_features, temporal_test, transaction_features, session_features, enhanced_session_features)

# Prepare for modeling
y_train = X_train_elite['DefaultLabel'].astype(int)
non_feature_cols = ['CustomerID', 'Week', 'DefaultLabel', 'first_txn', 'last_txn', 'first_session', 'last_session']
feature_cols = [col for col in X_train_elite.columns if col not in non_feature_cols]

X_train = X_train_elite[feature_cols]
X_test = X_test_elite[feature_cols]

print(f"✅ Elite feature matrix: {X_train.shape[1]} features, {X_train.shape[0]} samples")

# Ensure numeric types and handle infinite values
X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0).replace([np.inf, -np.inf], 0)

print("🔍 Performing STABLE feature selection...")

# STABLE FEATURE SELECTION
stable_selector = SelectFromModel(
    RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    threshold='median',  # More stable than top-k
    max_features=60
)
stable_selector.fit(X_train, y_train)

selected_mask = stable_selector.get_support()
selected_features = X_train.columns[selected_mask]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

print(f"🎯 Selected {len(selected_features)} stable features")

# STABLE THRESHOLD OPTIMIZATION
def stable_threshold_optimization(y_true, y_proba, n_bootstrap=50):
    """More stable threshold optimization with bootstrapping"""
    thresholds = np.linspace(0.2, 0.5, 80)  # Wider range, more points
    threshold_scores = []

    for threshold in thresholds:
        bootstrap_scores = []
        for _ in range(n_bootstrap):
            # Bootstrap sampling with fixed seed for each iteration
            indices = np.random.RandomState(42).choice(len(y_true), len(y_true), replace=True)
            y_true_boot = y_true.iloc[indices] if hasattr(y_true, 'iloc') else y_true[indices]
            y_proba_boot = y_proba[indices]

            y_pred_boot = (y_proba_boot > threshold).astype(int)
            score = f1_score(y_true_boot, y_pred_boot, average='macro')
            bootstrap_scores.append(score)

        threshold_scores.append(np.mean(bootstrap_scores))

    best_idx = np.argmax(threshold_scores)
    return thresholds[best_idx], threshold_scores[best_idx]

# STABLE PREPROCESSING PIPELINE
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

print("🤖 Training STABLE ensemble model...")

# Apply stable preprocessing
X_train_processed = preprocessing_pipeline.fit_transform(X_train_selected)
X_test_processed = preprocessing_pipeline.transform(X_test_selected)

# STABLE ENSEMBLE MODEL
stable_ensemble = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(
            n_estimators=400,
            max_depth=25,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            class_weight='balanced_subsample',
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        )),
        ('gb', GradientBoostingClassifier(
            n_estimators=250,
            max_depth=7,
            learning_rate=0.08,
            subsample=0.85,
            random_state=42
        ))
    ],
    voting='soft',
    weights=[0.65, 0.35]  # Fixed weights for stability
)

# Train the stable ensemble
stable_ensemble.fit(X_train_processed, y_train)

# Get probabilities
y_train_proba = stable_ensemble.predict_proba(X_train_processed)[:, 1]
y_test_proba = stable_ensemble.predict_proba(X_test_processed)[:, 1]

print("📊 Performing STABLE threshold optimization...")

# Find optimal threshold
optimal_threshold, optimal_f1 = stable_threshold_optimization(y_train, y_train_proba)

print(f"✅ Optimal threshold: {optimal_threshold:.3f}")
print(f"✅ Train F1 score: {optimal_f1:.4f}")

# Final predictions
final_predictions = (y_test_proba > optimal_threshold).astype(int)

# Create submission
submission_df = pd.DataFrame({
    'CustomerID': test_panel['CustomerID'],
    'Week': test_panel['Week'],
    'DefaultLabel': final_predictions
})

submission_filename = "retailbanking_challenge2_stable_predictions.csv"
submission_df.to_csv(submission_filename, index=False)

print(f"🎉 STABLE predictions saved: {len(final_predictions)} predictions")
print(f"   Default rate: {final_predictions.mean():.3f} ({final_predictions.sum()} defaults)")

# Model interpretation
def explain_predictions(model, feature_names, top_n=20):
    """Provide business-interpretable feature importance"""
    # For VotingClassifier, use the first estimator (RF) for feature importance
    if hasattr(model, 'estimators_'):
        rf_model = model.estimators_[0]
        importances = rf_model.feature_importances_
    else:
        importances = model.feature_importances_

    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(top_n)

    # Categorize features
    categories = {
        'Credit Risk': ['CreditScore', 'credit_utilization', 'high_risk_profile', 'credit_health'],
        'Payment Behavior': ['PaymentRatio', 'payment_trend', 'payment_volatility', 'payment_consistency'],
        'Spending Patterns': ['utilization_ma', 'transaction_risk_score', 'spending_volatility'],
        'Session Behavior': ['session_risk_score', 'financial_actions', 'sensitive_action_ratio', 'action_sequence'],
        'Behavioral Regimes': ['regime', 'deteriorating', 'persistence'],
        'Composite Scores': ['comprehensive_risk', 'final_risk', 'stress_score']
    }

    for feature in importance_df['feature']:
        category_found = False
        for category, keywords in categories.items():
            if any(keyword in feature for keyword in keywords):
                importance_df.loc[importance_df['feature'] == feature, 'category'] = category
                category_found = True
                break
        if not category_found:
            importance_df.loc[importance_df['feature'] == feature, 'category'] = 'Other'

    return importance_df

print("🔍 Top predictive features:")
feature_explanation = explain_predictions(stable_ensemble, selected_features)
print(feature_explanation[['feature', 'category', 'importance']].head(15))

# Submit predictions
try:
    from agentds import BenchmarkClient
    client = BenchmarkClient(api_key="adsb_E8N9aNAz2w1K7dNYT8BSMGsd_1760199769", team_name="synergy-minds")

    result = client.submit_prediction("Retailbanking", 2, submission_filename)

    if result['success']:
        print("🏆 STABLE Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        current_score = result['score']

        if current_score >= 0.9500:
            print("   🎉 EXCELLENT! Consistent elite performance achieved!")
        elif current_score >= 0.9450:
            print("   💪 Very good! Close to elite tier!")
        else:
            improvement_needed = 0.9500 - current_score
            print(f"   🎯 Need improvement: {improvement_needed:.4f} to reach elite tier")

        # Track improvements
        baseline_score = 0.4800
        improvement = current_score - baseline_score
        print(f"   📈 Overall improvement from baseline: +{improvement:.4f}")

    else:
        print("❌ Submission failed!")

except Exception as e:
    print(f"💥 Submission error: {e}")

print("\n💡 STABLE Strategy Features:")
print("   • Comprehensive random seed control")
print("   • Stable feature selection with median threshold")
print("   • Bootstrapped threshold optimization")
print("   • Fixed ensemble weights (no dynamic variability)")
print("   • Reproducible preprocessing pipeline")
print("   • Consistent data handling")
print("   • Enhanced model stability parameters")

# VALIDATION: Run multiple times to check consistency
print("\n🔍 Running stability validation (3 quick runs)...")
validation_scores = []

for i in range(3):
    set_all_seeds(42 + i)  # Different seeds for validation

    # Quick retrain with different seed to test stability
    val_ensemble = VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42+i)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42+i))
        ],
        voting='soft',
        weights=[0.65, 0.35]
    )

    val_ensemble.fit(X_train_processed, y_train)
    y_val_proba = val_ensemble.predict_proba(X_train_processed)[:, 1]
    val_threshold, val_f1 = stable_threshold_optimization(y_train, y_val_proba, n_bootstrap=10)
    validation_scores.append(val_f1)
    print(f"   Run {i+1}: F1 = {val_f1:.4f}, Threshold = {val_threshold:.3f}")

print(f"✅ Stability check - Score range: {max(validation_scores):.4f} - {min(validation_scores):.4f}")
print(f"   Consistency: \u00b1{np.std(validation_scores):.4f}")